# Create HATVP dataset

The first part creates the dataset from the base XML file and uploads it to the HUB. Remove the `if False` statements to run again if needed.  

## install libs

In [ ]:
!pip install datasets

## Load base declarations.xml file

In [ ]:
# slow option
# !wget https://www.hatvp.fr/livraison/merge/declarations.xml

# we get the XML from GDrive to speedup the process
main_file_path = "/content/drive/MyDrive/AI/HATVP/24-04-2024_datasets/declarations.xml"

## Create dataset with singular declarations

In [ ]:
import hashlib

def sha1_digest(input_string):
    # Convert the input string to bytes since hashlib works with bytes
    input_bytes = input_string.encode('utf-8')

    # Create a new SHA-1 hash object
    sha1 = hashlib.sha1()

    # Update the hash object with the input bytes
    sha1.update(input_bytes)

    # Get the hexadecimal digest of the hash
    sha1_hex_digest = sha1.hexdigest()

    return sha1_hex_digest

# Example usage:
input_string = "Hello, World!"
print("SHA-1 digest:", sha1_digest(input_string))

SHA-1 digest: 0a0a9f2a6772942557ab5355d76af442f8f65e01


In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm.auto import tqdm


def split_xml_documents(file_path):
    # Create a folder named "split_xml" if it doesn't exist
    if not os.path.exists("split_xml"):
        os.makedirs("split_xml")

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize a list to store XML documents as strings
    xml_strings = []

    # Iterate through each 'declaration' element and convert it to string
    for i, declaration in enumerate(tqdm(root.findall('declaration'), desc='extracting declarations')):
        # Convert the declaration element to string and append to the list
        single_text_declaration = ET.tostring(declaration, encoding='unicode')
        single_text_declaration = single_text_declaration.replace('\n', '').replace('\t', '')
        xml_strings.append([sha1_digest(input_string), single_text_declaration])

        # activate in case of debug
        # if i > 10:
        #   break

    return xml_strings


In [ ]:
create_dataset = True

In [ ]:
if create_dataset:
  declaration_list = split_xml_documents(main_file_path)

extracting declarations:   0%|          | 0/10944 [00:00<?, ?it/s]

In [ ]:
if create_dataset:
  len(declaration_list)

In [ ]:
import pandas as pd

if create_dataset:
  declaration_df = pd.DataFrame(declaration_list, columns=['xml_sha1','declaration_xml'])


In [ ]:
from datasets import Dataset

if create_dataset:
  declaration_ds = Dataset.from_pandas(declaration_df)

In [ ]:
if create_dataset:
  print(declaration_ds)

Dataset({
    features: ['xml_sha1', 'declaration_xml'],
    num_rows: 10944
})


## Upload dataset to HUB

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
if create_dataset:
  declaration_ds.push_to_hub("the-french-artist/hatvp_declarations_xml")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/346 [00:00<?, ?B/s]

In [ ]:
declaration_ds.to_pandas()

,xml_sha1,declaration_xml
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...
...,...,...
10939,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>15/02/2022 11:07:33</d...
10940,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/06/2021 22:39:18</d...
10941,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/12/2020 23:48:29</d...
10942,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>25/06/2021 13:29:08</d...


## Load dataset

In [ ]:
from datasets import load_dataset
declaration_ds = load_dataset("the-french-artist/hatvp_declarations_xml", split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
singular_test_declaration = declaration_ds.select(range(1)).to_pandas().declaration_xml.to_list()[0]

In [ ]:
singular_test_declaration

"<declaration><dateDepot>11/07/2022 15:40:13</dateDepot><uuid>4344aaa1-874d-4e6d-9b1a-45f7725b710c</uuid><origine>ADEL</origine><complete>true</complete><attachedFiles><attachedFiles><fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName><serverFileName /><base64EncodedContent /></attachedFiles></attachedFiles><declarationVersion>20171221</declarationVersion><activConsultantDto><neant>true</neant></activConsultantDto><activProfCinqDerniereDto><neant>true</neant></activProfCinqDerniereDto><activProfConjointDto><items><items><motif><id>CREATION</id><label /></motif><commentaire /><nomConjoint>            [Données non publiées]        </nomConjoint><employeurConjoint>CENTRE HOSPITALIER DU HAUT-BUGEY</employeurConjoint><activiteProf>Infirmière</activiteProf></items></items><neant>false</neant></activProfConjointDto><fonctionBenevoleDto><neant>true</neant></fonctionBenevoleDto><mandatElectifDto><items><items><motif><id>CREATION</id><label /></motif><commentaire>REVENUS NETS IMPOSABLES       

## Make objects

WARNING: each iteration is missing previously declared classes, we will need to make a final version that contains all code in one place.

### Incremental coding (all commented)

#### Base declaration

In [ ]:
# import xml.etree.ElementTree as ET

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#           Loaded from XPath: 'dateDepot'
#         - uuid: "Identifiant Universel Unique"
#           Loaded from XPath: 'uuid'
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#           Loaded from XPath: 'origine'
#         - complete: "Déclaration complète ou non"
#           Loaded from XPath: 'complete'
#         """
#         try:
#             # Parse the XML string into an ElementTree object
#             xml_tree = ET.fromstring(xml_string)
#             # Parse each attribute from the XML tree using the specified XPath
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#         except ET.ParseError as e:
#             # Handle parsing errors (e.g., malformed XML)
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             # Handle other generic exceptions
#             print(f"An error occurred: {e}")

#     def _get_xml_text(self, tree, xpath):
#         """ Helper function to extract text from the specified XPath or return None if not found """
#         element = tree.find(xpath)
#         if element is not None:
#             return element.text
#         else:
#             # Log a warning if the element is not found
#             print(f"Warning: No element found for XPath '{xpath}'. Returning None.")
#             return None

#     def print_info(self):
#         """
#         Print information about the Declaration.
#         """
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### Added AttachedFiles

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List

# class AttachedFiles:
#     def __init__(self, element):
#         """
#         Initialize the AttachedFiles class by parsing an XML element.

#         Attributes:
#         - file_name: "Nom du fichier de la pièce jointe"
#         - server_file_name: "Nom du fichier sur le serveur de la pièce jointe"
#         """
#         self.file_name = self._get_xml_text(element, 'fileName')
#         self.server_file_name = self._get_xml_text(element, 'serverFileName')

#     @staticmethod
#     def _get_xml_text(element, xpath):
#         """ Helper function to extract text from the specified XPath or return None if not found """
#         child = element.find(xpath)
#         if child is not None:
#             return child.text
#         else:
#             # Log a warning if the element is not found
#             print(f"Warning: No element found for XPath '{xpath}'. Returning None.")
#             return None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#         - uuid: "Identifiant Universel Unique"
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#         - complete: "Déclaration complète ou non"
#         - attached_files: List of AttachedFiles instances
#         """
#         try:
#             # Parse the XML string into an ElementTree object
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))

#         except ET.ParseError as e:
#             # Handle parsing errors (e.g., malformed XML)
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             # Handle other generic exceptions
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         """ Helper function to extract text from the specified XPath or return None if not found """
#         element = tree.find(xpath)
#         if element is not None:
#             return element.text
#         else:
#             # Log a warning if the element is not found
#             print(f"Warning: No element found for XPath '{xpath}'. Returning None.")
#             return None

#     def _parse_attached_files(self, attached_files_element):
#         """ Parse the attached files from the XML tree """
#         attached_files = []
#         if attached_files_element is not None:
#             for file_element in attached_files_element.findall('attachedFiles'):
#                 attached_files.append(AttachedFiles(file_element))
#         return attached_files

#     def print_info(self):
#         """
#         Print information about the Declaration.
#         """
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")
#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # This should be a valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### Adding Remuneration and ActivProfCinqDerniereDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class Remuneration:
#     def __init__(self, element):
#         """
#         Initialize the Remuneration class by parsing an XML element.

#         Attributes:
#         - type: "Brut ou Net"
#         - year: "Année pour la rémunération"
#         - amount: "Montant de la rémunération"
#         """
#         self.type = element.find('brutNet').text if element.find('brutNet') is not None else None
#         self.year = int(element.find('montant/annee').text) if element.find('montant/annee') is not None else None
#         self.amount = float(element.find('montant/montant').text) if element.find('montant/montant') is not None else None

# class ActivProfCinqDerniereDto:
#     def __init__(self, element):
#         """
#         Initialize the ActivProfCinqDerniereDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - description: "Description de l'activité"
#         - employer_name: "Nom de l'employeur pour l'activité"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.description = element.find('description').text if element.find('description') is not None else None
#         self.employer_name = element.find('employeur').text if element.find('employeur') is not None else None
#         self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#         - uuid: "Identifiant Universel Unique"
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#         - complete: "Déclaration complète ou non"
#         - attached_files: List of AttachedFiles instances
#         - professional_activities: List of ActivProfCinqDerniereDto instances
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None or activities_element.find('neant') is not None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]

#     def print_info(self):
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")
#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")
#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
#             for rem in activity.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### Add MandatElectifDto and Update Remuneration class

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List

# class Remuneration:
#     def __init__(self, element):
#         """
#         Initialize the Remuneration class by parsing an XML element.

#         Attributes:
#         - type: "Brut ou Net"
#         - year: "Année pour la rémunération"
#         - amount: "Montant de la rémunération"
#         """
#         self.type = element.find('brutNet').text if element.find('brutNet') is not None else None
#         self.year = []
#         self.amount = []
#         # Handling multiple 'montant' elements properly
#         montants = element.findall('montant/montant')  # Adjusted XPath to access nested montant tags
#         for montant in montants:
#             year = montant.find('annee')
#             amount = montant.find('montant')
#             if year is not None and amount is not None:
#                 self.year.append(int(year.text))
#                 self.amount.append(self.parse_amount(amount.text))

#     @staticmethod
#     def parse_amount(amount_str):
#         """ Parse and clean the amount string to handle non-numeric characters and convert it to float """
#         cleaned_amount = ''.join(c for c in amount_str if c.isdigit() or c == '.')
#         return float(cleaned_amount)

# class MandatElectifDto:
#     def __init__(self, element):
#         """
#         Initialize the MandatElectifDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - description: "Identification de la fonction élective ou du mandat"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.description = element.find('descriptionMandat').text if element.find('descriptionMandat') is not None else None
#         self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None


# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#         - uuid: "Identifiant Universel Unique"
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#         - complete: "Déclaration complète ou non"
#         - attached_files: List of AttachedFiles instances
#         - professional_activities: List of ActivProfCinqDerniereDto instances
#         - elective_mandates: List of MandatElectifDto instances
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def print_info(self):
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")
#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")
#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
#             for rem in activity.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")
#         print("Elective Mandates:")
#         for mandate in self.elective_mandates:
#             print(f"ID: {mandate.id}, Label: {mandate.label}, Comment: {mandate.comment}, Conserved: {mandate.conserved}, Description: {mandate.description}, Start Date: {mandate.start_date}, End Date: {mandate.end_date}")
#             for rem in mandate.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### Add GeneralInfo

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class GeneralInfo:
#     def __init__(self, element):
#         """
#         Initialize the GeneralInfo class by parsing an XML element.

#         Attributes: Various details about the declaration and the declarant as specified.
#         """
#         self.type_declaration_id = element.find('typeDeclaration/id').text if element.find('typeDeclaration/id') is not None else None
#         self.type_declaration_label = element.find('typeDeclaration/label').text if element.find('typeDeclaration/label') is not None else None
#         self.mandat_label = element.find('mandat/label').text if element.find('mandat/label') is not None else None
#         self.type_mandat = element.find('qualiteMandat/typeMandat').text if element.find('qualiteMandat/typeMandat') is not None else None
#         self.cod_categorie_mandat = element.find('qualiteMandat/codCategorieMandat').text if element.find('qualiteMandat/codCategorieMandat') is not None else None
#         self.nom_categorie_mandat = element.find('qualiteMandat/nomCategorieMandat').text if element.find('qualiteMandat/nomCategorieMandat') is not None else None
#         self.cod_type_mandat_fichier = element.find('qualiteMandat/codTypeMandatFichier').text if element.find('qualiteMandat/codTypeMandatFichier') is not None else None
#         self.label_type_mandat = element.find('qualiteMandat/labelTypeMandat').text if element.find('qualiteMandat/labelTypeMandat') is not None else None
#         self.label_organe = element.find('qualiteMandat/labelOrgane').text if element.find('qualiteMandat/labelOrgane') is not None else None
#         self.code_liste_organe = element.find('qualiteMandat/codeListeOrgane').text if element.find('qualiteMandat/codeListeOrgane') is not None else None
#         self.texte_aide = element.find('qualiteMandat/texteAide').text if element.find('qualiteMandat/texteAide') is not None else None
#         self.code_organe = element.find('organe/codeListeOrgane').text if element.find('organe/codeListeOrgane') is not None else None
#         self.nom_organe = element.find('organe/nomListeOrgane').text if element.find('organe/nomListeOrgane') is not None else None
#         self.label_organe = element.find('organe/labelOrgane').text if element.find('organe/labelOrgane') is not None else None
#         self.label_declaration = element.find('organe/labelDeclaration').text if element.find('organe/labelDeclaration') is not None else None
#         self.qualite_declarant = element.find('qualiteDeclarant').text if element.find('qualiteDeclarant') is not None else None
#         self.qualite_declarant_pdf = element.find('qualiteDeclarantForPDF').text if element.find('qualiteDeclarantForPDF') is not None else None
#         self.date_debut_mandat = element.find('dateDebutMandat').text if element.find('dateDebutMandat') is not None else None
#         self.date_fin_mandat = element.find('dateFinMandat').text if element.find('dateFinMandat') is not None else None
#         self.date_derniere_declaration = element.find('dateDernDeclar').text if element.find('dateDernDeclar') is not None else None
#         self.regime_matrimonial = element.find('regimeMatrimonial').text if element.find('regimeMatrimonial') is not None else None
#         self.regime_matrimonial_comments = element.find('regimeMatrimonialComments').text if element.find('regimeMatrimonialComments') is not None else None
#         self.nom_societe = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.nom_societe_mere = element.find('nomSocieteMere').text if element.find('nomSocieteMere') is not None else None
#         self.chiffre_affaire = element.find('chiffreAffaire').text if element.find('chiffreAffaire') is not None else None
#         self.nombre_logements = element.find('nbLogements').text if element.find('nbLogements') is not None else None
#         self.declarant_civilite = element.find('declarant/civilite').text if element.find('declarant/civilite') is not None else None
#         self.declarant_nom = element.find('declarant/nom').text if element.find('declarant/nom') is not None else None
#         self.declarant_prenom = element.find('declarant/prenom').text if element.find('declarant/prenom') is not None else None
#         self.declarant_email = element.find('declarant/email').text if element.find('declarant/email') is not None else None
#         self.declarant_telephone = element.find('declarant/telephone').text if element.find('declarant/telephone') is not None else None
#         self.declarant_date_naissance = element.find('declarant/dateNaissance').text if element.find('declarant/dateNaissance') is not None else None
#         self.declarant_voie = element.find('declarant/adresseDec/voie').text if element.find('declarant/adresseDec/voie') is not None else None
#         self.declarant_complement = element.find('declarant/adresseDec/complement').text if element.find('declarant/adresseDec/complement') is not None else None
#         self.declarant_code_postal = element.find('declarant/adresseDec/codePostal').text if element.find('declarant/adresseDec/codePostal') is not None else None
#         self.declarant_ville = element.find('declarant/adresseDec/ville').text if element.find('declarant/adresseDec/ville') is not None else None
#         self.declarant_pays = element.find('declarant/adresseDec/pays').text if element.find('declarant/adresseDec/pays') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#         - uuid: "Identifiant Universel Unique"
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#         - complete: "Déclaration complète ou non"
#         - attached_files: List of AttachedFiles instances
#         - professional_activities: List of ActivProfCinqDerniereDto instances
#         - elective_mandates: List of MandatElectifDto instances
#         - general_info: General information about the declarant and the declaration
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print information about the Declaration.
#         """
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")
#         print(f"General Information:")
#         if self.general_info:
#             print(f"Type of Declaration ID: {self.general_info.type_declaration_id}, Label: {self.general_info.type_declaration_label}")
#             print(f"Label Mandate: {self.general_info.mandat_label}, Type of Mandate: {self.general_info.type_mandat}")
#             print(f"Cod Category Mandate: {self.general_info.cod_categorie_mandat}, Name Category Mandate: {self.general_info.nom_categorie_mandat}")
#             # Add more prints here for other fields as needed
#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")
#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
#             for rem in activity.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")
#         print("Elective Mandates:")
#         for mandate in self.elective_mandates:
#             print(f"ID: {mandate.id}, Label: {mandate.label}, Comment: {mandate.comment}, Conserved: {mandate.conserved}, Description: {mandate.description}, Start Date: {mandate.start_date}, End Date: {mandate.end_date}")
#             for rem in mandate.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add ActivProfConjointDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ActivProfConjointDto:
#     def __init__(self, element):
#         """
#         Initialize the ActivProfConjointDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - spouse_name: "Nom du conjoint"
#         - employer: "Employeur du conjoint"
#         - activity_description: "Description de l’activité professionnelle du conjoint"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.spouse_name = element.find('nomConjoint').text if element.find('nomConjoint') is not None else None
#         self.employer = element.find('employeurConjoint').text if element.find('employeurConjoint') is not None else None
#         self.activity_description = element.find('activiteProf').text if element.find('activiteProf') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes:
#         - date_depot: "Date de dépôt de la déclaration sur le téléservice"
#         - uuid: "Identifiant Universel Unique"
#         - origine: "Téléservice utilisé pour déposer la déclaration"
#         - complete: "Déclaration complète ou non"
#         - attached_files: List of AttachedFiles instances
#         - professional_activities: List of ActivProfCinqDerniereDto instances
#         - elective_mandates: List of MandatElectifDto instances
#         - general_info: General information about the declarant and the declaration
#         - spouse_activities: List of ActivProfConjointDto instances
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print information about the Declaration.
#         """
#         print("Declaration Information:")
#         print(f"Date de dépôt: {self.date_depot}")
#         print(f"UUID: {self.uuid}")
#         print(f"Origine: {self.origine}")
#         print(f"Complète: {self.complete}")
#         print("General Information:")
#         if self.general_info:
#             print(f"Type of Declaration ID: {self.general_info.type_declaration_id}, Label: {self.general_info.type_declaration_label}")
#             # Additional prints for general information can be listed here.
#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")
#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}")
#             # More details can be printed as needed.
#         print("Spouse's Professional Activities:")
#         for activity in self.spouse_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Spouse Name: {activity.spouse_name}, Employer: {activity.employer}, Activity Description: {activity.activity_description}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add ParticipationDirigeantDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ParticipationDirigeantDto:
#     def __init__(self, element):
#         """
#         Initialize the ParticipationDirigeantDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - company_name: "Identification de l’organisme public ou privé ou de la société"
#         - activity_description: "Description de l’activité"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.activity_description = element.find('activite').text if element.find('activite') is not None else None
#         self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including professional and directorial activities.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]
#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print information about the Declaration, including details of directorial participations.
#         """
#         print("Declaration Information:")
#         # Print various details as needed, including directorial participations
#         for participation in self.director_participations:
#             print(f"Participation ID: {participation.id}, Label: {participation.label}, Comment: {participation.comment}, Conserved: {participation.conserved}, Company Name: {participation.company_name}, Description: {participation.activity_description}, Start Date: {participation.start_date}, End Date: {participation.end_date}")
#             for rem in participation.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add ParticipationFinanciereDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ParticipationFinanciereDto:
#     def __init__(self, element):
#         """
#         Initialize the ParticipationFinanciereDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - company_name: "Identification de la société"
#         - evaluation: "Évaluation de la participation financière"
#         - remuneration: "Rémunération ou gratification perçue"
#         - capital_held: "Pourcentage du capital détenu"
#         - number_of_shares: "Nombre de parts"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.evaluation = element.find('evaluation').text if element.find('evaluation') is not None else None
#         self.remuneration = element.find('remuneration').text if element.find('remuneration') is not None else None
#         self.capital_held = element.find('capitalDetenu').text if element.find('capitalDetenu') is not None else None
#         self.number_of_shares = element.find('nombreParts').text if element.find('nombreParts') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including financial participations.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]
#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including financial participations.
#         """
#         print("Declaration Information:")
#         # Print other details as needed
#         print("Financial Participations:")
#         for participation in self.financial_participations:
#             print(f"ID: {participation.id}, Label: {participation.label}, Comment: {participation.comment}, Company Name: {participation.company_name}, Evaluation: {participation.evaluation}, Remuneration: {participation.remuneration}, Capital Held: {participation.capital_held}, Number of Shares: {participation.number_of_shares}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add activCollaborateursDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ActivCollaborateursDto:
#     def __init__(self, element):
#         """
#         Initialize the ActivCollaborateursDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - collaborator_name: "Nom et prénom du collaborateur"
#         - employer: "Identification des autres employeurs ou structures sociales d’emploi"
#         - activity_description: "Description des activités professionnelles"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.collaborator_name = element.find('nom').text if element.find('nom') is not None else None
#         self.employer = element.find('employeur').text if element.find('employeur') is not None else None
#         self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including collaborators.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#             self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]
#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

#     def _parse_parliamentary_collaborators(self, collaborators_element):
#         if collaborators_element is None:
#             return []  # No collaborators to report
#         return [ActivCollaborateursDto(item) for item in collaborators_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including parliamentary collaborators.
#         """
#         print("Declaration Information:")
#         # Print other details as needed
#         print("Parliamentary Collaborators:")
#         for collaborator in self.parliamentary_collaborators:
#             print(f"ID: {collaborator.id}, Label: {collaborator.label}, Comment: {collaborator.comment}, Collaborator Name: {collaborator.collaborator_name}, Employer: {collaborator.employer}, Activity Description: {collaborator.activity_description}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add ObservationInteretDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ObservationInteretDto:
#     def __init__(self, element):
#         """
#         Initialize the ObservationInteretDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - content: "Observations du déclarant"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.content = element.find('contenu').text if element.find('contenu') is not None else None

# class ActivCollaborateursDto:
#     def __init__(self, element):
#         """
#         Initialize the ActivCollaborateursDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - collaborator_name: "Nom et prénom du collaborateur"
#         - employer: "Identification des autres employeurs ou structures sociales d’emploi"
#         - activity_description: "Description des activités professionnelles"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.collaborator_name = element.find('nom').text if element.find('nom') is not None else None
#         self.employer = element.find('employeur').text if element.find('employeur') is not None else None
#         self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including observations.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#             self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
#             self.observations = self._parse_observations(xml_tree.find('observationInteretDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]
#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

#     def _parse_parliamentary_collaborators(self, collaborators_element):
#         if collaborators_element is None:
#             return []  # No collaborators to report
#         return [ActivCollaborateursDto(item) for item in collaborators_element.findall('items/items')]

#     def _parse_observations(self, observation_element):
#         if observation_element is None:
#             return []  # No observations to report
#         return [ObservationInteretDto(item) for item in observation_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including observations.
#         """
#         print("Declaration Information:")
#         # Print other sections as needed

#         print("Observations:")
#         for observation in self.observations:
#             print(f"ID: {observation.id}, Label: {observation.label}, Comment: {observation.comment}, Content: {observation.content}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### add activConsultantDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class ActivConsultantDto:
#     def __init__(self, element):
#         """
#         Initialize the ActivConsultantDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - employer_name: "Nom de l'employeur pour l'activité"
#         - activity_description: "Description de l'activité"
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         - remuneration_type: "Brut ou Net"
#         - remuneration_amount: "Montant de la rémunération"
#         - remuneration_year: "Année pour la rémunération"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.employer_name = element.find('nomEmployeur').text if element.find('nomEmployeur') is not None else None
#         self.activity_description = element.find('description').text if element.find('description') is not None else None
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None
#         remuneration_element = element.find('remuneration')
#         self.remuneration_type = remuneration_element.find('brutNet').text if remuneration_element is not None else None
#         self.remuneration_amount = remuneration_element.find('montant/montant').text if remuneration_element is not None else None
#         self.remuneration_year = remuneration_element.find('montant/annee').text if remuneration_element is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including consultant activities.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#             self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
#             self.observations = self._parse_observations(xml_tree.find('observationInteretDto'))
#             self.consultant_activities = self._parse_consultant_activities(xml_tree.find('activConsultantDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")


#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]

#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

#     def _parse_parliamentary_collaborators(self, collaborators_element):
#         if collaborators_element is None:
#             return []  # No collaborators to report
#         return [ActivCollaborateursDto(item) for item in collaborators_element.findall('items/items')]

#     def _parse_observations(self, observation_element):
#         if observation_element is None:
#             return []  # No observations to report
#         return [ObservationInteretDto(item) for item in observation_element.findall('items/items')]


#     def _parse_consultant_activities(self, consultant_element):
#         if consultant_element is None:
#             return []  # No consultant activities to report
#         return [ActivConsultantDto(item) for item in consultant_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including consultant activities.
#         """
#         print("Declaration Information:")
#         # Print details of consultant activities and other sections
#         print("Consultant Activities:")
#         for activity in self.consultant_activities:
#             print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Employer: {activity.employer_name}, Description: {activity.activity_description}, Start Date: {activity.start_date}, End Date: {activity.end_date}, Remuneration Type: {activity.remuneration_type}, Amount: {activity.remuneration_amount}, Year: {activity.remuneration_year}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


#### fonctionBenevoleDto

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class FonctionBenevoleDto:
#     def __init__(self, element):
#         """
#         Initialize the FonctionBenevoleDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - organization_name: "Nom et objet de la structure ou de la personne morale"
#         - activity_description: "Description des activités et responsabilités exercées"
#         """
#         self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.organization_name = element.find('nomStructure').text if element.find('nomStructure') is not None else None
#         self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

# class Declaration:
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including voluntary functions.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#             self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
#             self.observations = self._parse_observations(xml_tree.find('observationInteretDto'))
#             self.volunteer_functions = self._parse_volunteer_functions(xml_tree.find('fonctionBenevoleDto'))
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")


#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]

#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

#     def _parse_parliamentary_collaborators(self, collaborators_element):
#         if collaborators_element is None:
#             return []  # No collaborators to report
#         return [ActivCollaborateursDto(item) for item in collaborators_element.findall('items/items')]

#     def _parse_observations(self, observation_element):
#         if observation_element is None:
#             return []  # No observations to report
#         return [ObservationInteretDto(item) for item in observation_element.findall('items/items')]

#     def _parse_volunteer_functions(self, volunteer_element):
#         if volunteer_element is None:
#             return []  # No volunteer functions to report
#         return [FonctionBenevoleDto(item) for item in volunteer_element.findall('items/items')]

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including voluntary functions.
#         """
#         print("Declaration Information:")
#         # Print other sections as needed
#         print("Volunteer Functions:")
#         for function in self.volunteer_functions:
#             print(f"ID: {function.id}, Label: {function.label}, Comment: {function.comment}, Conserved: {function.conserved}, Organization Name: {function.organization_name}, Activity Description: {function.activity_description}")

# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


### Complete classes  

We make classes from the XML file.

In [ ]:
import xml.etree.ElementTree as ET
from typing import List, Optional

class AttachedFiles:
    def __init__(self, element):
        """
        Initialize the AttachedFiles class by parsing an XML element.

        Attributes:
        - file_name: "Nom du fichier de la pièce jointe"
        - server_file_name: "Nom du fichier sur le serveur de la pièce jointe"
        """
        self.file_name = self._get_xml_text(element, 'fileName')
        self.server_file_name = self._get_xml_text(element, 'serverFileName')

    @staticmethod
    def _get_xml_text(element, xpath):
        """ Helper function to extract text from the specified XPath or return None if not found """
        child = element.find(xpath)
        if child is not None:
            return child.text
        else:
            # Log a warning if the element is not found
            print(f"Warning: No element found for XPath '{xpath}'. Returning None.")
            return None

class ActivConsultantDto:
    def __init__(self, element):
        """
        Initialize the ActivConsultantDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - conserved: "Indique si l'activité est conservée"
        - employer_name: "Nom de l'employeur pour l'activité"
        - activity_description: "Description de l'activité"
        - start_date: "Date de début de l'activité"
        - end_date: "Date de fin de l'activité"
        - remuneration_type: "Brut ou Net"
        - remuneration_amount: "Montant de la rémunération"
        - remuneration_year: "Année pour la rémunération"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
        self.employer_name = element.find('nomEmployeur').text if element.find('nomEmployeur') is not None else None
        self.activity_description = element.find('description').text if element.find('description') is not None else None
        self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
        self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None
        remuneration_element = element.find('remuneration')
        if remuneration_element is None:
          self.remuneration_type = None
          self.remuneration_amount = None
          self.remuneration_year = None
        self.remuneration_type = remuneration_element.find('brutNet').text if remuneration_element.find('brutNet') is not None else None
        self.remuneration_amount = remuneration_element.find('montant/montant').text if remuneration_element.find('montant/montant') is not None else None
        self.remuneration_year = remuneration_element.find('montant/annee').text if remuneration_element.find('montant/annee') is not None else None

class FonctionBenevoleDto:
    def __init__(self, element):
        """
        Initialize the FonctionBenevoleDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - conserved: "Indique si l'activité est conservée"
        - organization_name: "Nom et objet de la structure ou de la personne morale"
        - activity_description: "Description des activités et responsabilités exercées"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
        self.organization_name = element.find('nomStructure').text if element.find('nomStructure') is not None else None
        self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

class ActivProfCinqDerniereDto:
    def __init__(self, element):
        """
        Initialize the ActivProfCinqDerniereDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - conserved: "Indique si l'activité est conservée"
        - description: "Description de l'activité"
        - employer_name: "Nom de l'employeur pour l'activité"
        - remuneration: List of Remuneration objects
        - start_date: "Date de début de l'activité"
        - end_date: "Date de fin de l'activité"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
        self.description = element.find('description').text if element.find('description') is not None else None
        self.employer_name = element.find('employeur').text if element.find('employeur') is not None else None
        self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')] if element.findall('remuneration') is not None else []
        self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
        self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

class Remuneration:
    def __init__(self, element):
        """
        Initialize the Remuneration class by parsing an XML element.

        Attributes:
        - type: "Brut ou Net"
        - year: "Année pour la rémunération"
        - amount: "Montant de la rémunération"
        """
        self.type = element.find('brutNet').text if element.find('brutNet') is not None else None
        self.year = []
        self.amount = []
        # Handling multiple 'montant' elements properly
        if element.findall('montant/montant') is not None:
          montants = element.findall('montant/montant')  # Adjusted XPath to access nested montant tags
          for montant in montants:
              year = montant.find('annee')
              amount = montant.find('montant')
              if (year.text is not None) and (amount.text is not None):
                  self.year.append(int(year.text))
                  self.amount.append(self.parse_amount(amount.text))

    @staticmethod
    def parse_amount(amount_str):
        """ Parse and clean the amount string to handle non-numeric characters and convert it to float """
        cleaned_amount = ''.join(c for c in amount_str if c.isdigit() or c == '.')
        return float(cleaned_amount)

class MandatElectifDto:
    def __init__(self, element):
        """
        Initialize the MandatElectifDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - conserved: "Indique si l'activité est conservée"
        - description: "Identification de la fonction élective ou du mandat"
        - remuneration: List of Remuneration objects
        - start_date: "Date de début de l'activité"
        - end_date: "Date de fin de l'activité"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
        self.description = element.find('descriptionMandat').text if element.find('descriptionMandat') is not None else None
        self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')]
        self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
        self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None



class GeneralInfo:
    def __init__(self, element):
        """
        Initialize the GeneralInfo class by parsing an XML element.

        Attributes: Various details about the declaration and the declarant as specified.
        """
        self.type_declaration_id = element.find('typeDeclaration/id').text if element.find('typeDeclaration/id') is not None else None
        self.type_declaration_label = element.find('typeDeclaration/label').text if element.find('typeDeclaration/label') is not None else None
        self.mandat_label = element.find('mandat/label').text if element.find('mandat/label') is not None else None
        self.type_mandat = element.find('qualiteMandat/typeMandat').text if element.find('qualiteMandat/typeMandat') is not None else None
        self.cod_categorie_mandat = element.find('qualiteMandat/codCategorieMandat').text if element.find('qualiteMandat/codCategorieMandat') is not None else None
        self.nom_categorie_mandat = element.find('qualiteMandat/nomCategorieMandat').text if element.find('qualiteMandat/nomCategorieMandat') is not None else None
        self.cod_type_mandat_fichier = element.find('qualiteMandat/codTypeMandatFichier').text if element.find('qualiteMandat/codTypeMandatFichier') is not None else None
        self.label_type_mandat = element.find('qualiteMandat/labelTypeMandat').text if element.find('qualiteMandat/labelTypeMandat') is not None else None
        self.label_organe = element.find('qualiteMandat/labelOrgane').text if element.find('qualiteMandat/labelOrgane') is not None else None
        self.code_liste_organe = element.find('qualiteMandat/codeListeOrgane').text if element.find('qualiteMandat/codeListeOrgane') is not None else None
        self.texte_aide = element.find('qualiteMandat/texteAide').text if element.find('qualiteMandat/texteAide') is not None else None
        self.code_organe = element.find('organe/codeListeOrgane').text if element.find('organe/codeListeOrgane') is not None else None
        self.nom_organe = element.find('organe/nomListeOrgane').text if element.find('organe/nomListeOrgane') is not None else None
        self.label_organe = element.find('organe/labelOrgane').text if element.find('organe/labelOrgane') is not None else None
        self.label_declaration = element.find('organe/labelDeclaration').text if element.find('organe/labelDeclaration') is not None else None
        self.qualite_declarant = element.find('qualiteDeclarant').text if element.find('qualiteDeclarant') is not None else None
        self.qualite_declarant_pdf = element.find('qualiteDeclarantForPDF').text if element.find('qualiteDeclarantForPDF') is not None else None
        self.date_debut_mandat = element.find('dateDebutMandat').text if element.find('dateDebutMandat') is not None else None
        self.date_fin_mandat = element.find('dateFinMandat').text if element.find('dateFinMandat') is not None else None
        self.date_derniere_declaration = element.find('dateDernDeclar').text if element.find('dateDernDeclar') is not None else None
        self.regime_matrimonial = element.find('regimeMatrimonial').text if element.find('regimeMatrimonial') is not None else None
        self.regime_matrimonial_comments = element.find('regimeMatrimonialComments').text if element.find('regimeMatrimonialComments') is not None else None
        self.nom_societe = element.find('nomSociete').text if element.find('nomSociete') is not None else None
        self.nom_societe_mere = element.find('nomSocieteMere').text if element.find('nomSocieteMere') is not None else None
        self.chiffre_affaire = element.find('chiffreAffaire').text if element.find('chiffreAffaire') is not None else None
        self.nombre_logements = element.find('nbLogements').text if element.find('nbLogements') is not None else None
        self.declarant_civilite = element.find('declarant/civilite').text if element.find('declarant/civilite') is not None else None
        self.declarant_nom = element.find('declarant/nom').text if element.find('declarant/nom') is not None else None
        self.declarant_prenom = element.find('declarant/prenom').text if element.find('declarant/prenom') is not None else None
        self.declarant_email = element.find('declarant/email').text if element.find('declarant/email') is not None else None
        self.declarant_telephone = element.find('declarant/telephone').text if element.find('declarant/telephone') is not None else None
        self.declarant_date_naissance = element.find('declarant/dateNaissance').text if element.find('declarant/dateNaissance') is not None else None
        self.declarant_voie = element.find('declarant/adresseDec/voie').text if element.find('declarant/adresseDec/voie') is not None else None
        self.declarant_complement = element.find('declarant/adresseDec/complement').text if element.find('declarant/adresseDec/complement') is not None else None
        self.declarant_code_postal = element.find('declarant/adresseDec/codePostal').text if element.find('declarant/adresseDec/codePostal') is not None else None
        self.declarant_ville = element.find('declarant/adresseDec/ville').text if element.find('declarant/adresseDec/ville') is not None else None
        self.declarant_pays = element.find('declarant/adresseDec/pays').text if element.find('declarant/adresseDec/pays') is not None else None

class ActivProfConjointDto:
    def __init__(self, element):
        """
        Initialize the ActivProfConjointDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - spouse_name: "Nom du conjoint"
        - employer: "Employeur du conjoint"
        - activity_description: "Description de l’activité professionnelle du conjoint"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.spouse_name = element.find('nomConjoint').text if element.find('nomConjoint') is not None else None
        self.employer = element.find('employeurConjoint').text if element.find('employeurConjoint') is not None else None
        self.activity_description = element.find('activiteProf').text if element.find('activiteProf') is not None else None

class ParticipationDirigeantDto:
    def __init__(self, element):
        """
        Initialize the ParticipationDirigeantDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - conserved: "Indique si l'activité est conservée"
        - company_name: "Identification de l’organisme public ou privé ou de la société"
        - activity_description: "Description de l’activité"
        - remuneration: List of Remuneration objects
        - start_date: "Date de début de l'activité"
        - end_date: "Date de fin de l'activité"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
        self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
        self.activity_description = element.find('activite').text if element.find('activite') is not None else None
        self.remuneration = [Remuneration(rem) for rem in element.findall('remuneration')]
        self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
        self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

class ParticipationFinanciereDto:
    def __init__(self, element):
        """
        Initialize the ParticipationFinanciereDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - company_name: "Identification de la société"
        - evaluation: "Évaluation de la participation financière"
        - remuneration: "Rémunération ou gratification perçue"
        - capital_held: "Pourcentage du capital détenu"
        - number_of_shares: "Nombre de parts"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
        self.evaluation = element.find('evaluation').text if element.find('evaluation') is not None else None
        self.remuneration = element.find('remuneration').text if element.find('remuneration') is not None else None
        self.capital_held = element.find('capitalDetenu').text if element.find('capitalDetenu') is not None else None
        self.number_of_shares = element.find('nombreParts').text if element.find('nombreParts') is not None else None

class ActivCollaborateursDto:
    def __init__(self, element):
        """
        Initialize the ActivCollaborateursDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - collaborator_name: "Nom et prénom du collaborateur"
        - employer: "Identification des autres employeurs ou structures sociales d’emploi"
        - activity_description: "Description des activités professionnelles"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.collaborator_name = element.find('nom').text if element.find('nom') is not None else None
        self.employer = element.find('employeur').text if element.find('employeur') is not None else None
        self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

class ObservationInteretDto:
    def __init__(self, element):
        """
        Initialize the ObservationInteretDto class by parsing an XML element.

        Attributes:
        - id: "ID of the motif"
        - label: "Label of the motif"
        - comment: "Commentaire du déclarant sur la rubrique"
        - content: "Observations du déclarant"
        """
        self.id = element.find('motif/id').text if element.find('motif/id') is not None else None
        self.label = element.find('motif/label').text if element.find('motif/label') is not None else None
        self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
        self.content = element.find('contenu').text if element.find('contenu') is not None else None

class Declaration:
    def __init__(self, xml_string):
        """
        Initialize the Declaration class by parsing an XML string.

        Attributes: Various details about the declaration including observations.
        """

        debug = False

        try:
            xml_tree = ET.fromstring(xml_string)
            if debug:
              print('done 1')
            self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
            if debug:
              print('done 2')
            self.uuid = self._get_xml_text(xml_tree, 'uuid')
            if debug:
              print('done 3')
            self.origine = self._get_xml_text(xml_tree, 'origine')
            if debug:
              print('done 4')
            self.complete = self._get_xml_text(xml_tree, 'complete')
            if debug:
              print('done 5')
            self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
            if debug:
              print('done 6')
            self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
            if debug:
              print('done 7')
            self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
            if debug:
              print('done 8')
            self.general_info = GeneralInfo(xml_tree.find('general')) if xml_tree.find('general') is not None else None
            if debug:
              print('done 9')
            self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
            if debug:
              print('done 10')
            self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
            if debug:
              print('done 11')
            self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
            if debug:
              print('done 12')
            self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
            if debug:
              print('done 13')
            self.observations = self._parse_observations(xml_tree.find('observationInteretDto'))
            if debug:
              print('done 14')
            self.consultant_activities = self._parse_consultant_activities(xml_tree.find('activConsultantDto'))
            if debug:
              print('done 15')
            self.volunteer_functions = self._parse_volunteer_functions(xml_tree.find('fonctionBenevoleDto'))
            if debug:
              print('done 16')
        except ET.ParseError as e:
            print(f"Error parsing XML: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

    @staticmethod
    def _get_xml_text(tree, xpath):
        element = tree.find(xpath)
        return element.text if element is not None else None

    def _parse_attached_files(self, attached_files_element):
        return [AttachedFiles(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

    def _parse_professional_activities(self, activities_element):
        if activities_element is None:
            return []  # No activities to report or section is empty
        return [ActivProfCinqDerniereDto(item) for item in activities_element.findall('items/items')]


    def _parse_elective_mandates(self, mandates_element):
        if mandates_element is None or mandates_element.findall('items/items'):
            return []  # No mandates to report or section is empty
        return [MandatElectifDto(item) for item in mandates_element.findall('items/items')]

    def _parse_spouse_activities(self, activities_element):
        if activities_element is None:
            return []  # No activities to report or section is empty
        return [ActivProfConjointDto(item) for item in activities_element.findall('items/items')]

    def _parse_director_participations(self, director_element):
        if director_element is None:
            return []  # No participations to report
        return [ParticipationDirigeantDto(item) for item in director_element.findall('items/items')]

    def _parse_financial_participations(self, financial_element):
        if financial_element is None:
            return []  # No financial participations to report
        return [ParticipationFinanciereDto(item) for item in financial_element.findall('items/items')]

    def _parse_parliamentary_collaborators(self, collaborators_element):
        if collaborators_element is None:
            return []  # No collaborators to report
        return [ActivCollaborateursDto(item) for item in collaborators_element.findall('items/items')]

    def _parse_observations(self, observation_element):
        if observation_element is None:
            return []  # No observations to report
        return [ObservationInteretDto(item) for item in observation_element.findall('items/items')]

    def _parse_consultant_activities(self, consultant_element):
        if consultant_element is None:
            return []  # No consultant activities to report
        return [ActivConsultantDto(item) for item in consultant_element.findall('items/items')]


    def _parse_volunteer_functions(self, volunteer_element):
        if volunteer_element is None:
            return []  # No volunteer functions to report
        return [FonctionBenevoleDto(item) for item in volunteer_element.findall('items/items')]


    def print_info(self):
        """
        Print detailed information about the Declaration, including observations.
        """
        print("Declaration Information:")
        print()
        print(f"General Information:")
        if self.general_info:
            print(f"Type of Declaration ID: {self.general_info.type_declaration_id}, Label: {self.general_info.type_declaration_label}")
            print(f"Label Mandate: {self.general_info.mandat_label}, Type of Mandate: {self.general_info.type_mandat}")
            print(f"Cod Category Mandate: {self.general_info.cod_categorie_mandat}, Name Category Mandate: {self.general_info.nom_categorie_mandat}")
            print(f"Cod Type Mandate File: {self.general_info.cod_type_mandat_fichier}, Label Type Mandate: {self.general_info.label_type_mandat}")
            print(f"Label Organ: {self.general_info.label_organe}, Code List Organ: {self.general_info.code_liste_organe}")
            print(f"Help Text: {self.general_info.texte_aide}")
            print(f"Organ Code: {self.general_info.code_organe}, Organ Name: {self.general_info.nom_organe}, Organ Label: {self.general_info.label_organe}, Declaration Label: {self.general_info.label_declaration}")
            print(f"Declarant Quality: {self.general_info.qualite_declarant}, Declarant Quality for PDF: {self.general_info.qualite_declarant_pdf}")
            print(f"Start Date Mandate: {self.general_info.date_debut_mandat}, End Date Mandate: {self.general_info.date_fin_mandat}, Last Declaration Date: {self.general_info.date_derniere_declaration}")
            print(f"Marital Regime: {self.general_info.regime_matrimonial}, Marital Regime Comments: {self.general_info.regime_matrimonial_comments}")
            print(f"Company Name: {self.general_info.nom_societe}, Parent Company Name: {self.general_info.nom_societe_mere}, Turnover: {self.general_info.chiffre_affaire}, Number of Dwellings: {self.general_info.nombre_logements}")
            print(f"Declarant Civility: {self.general_info.declarant_civilite}, Declarant Name: {self.general_info.declarant_nom}, Declarant First Name: {self.general_info.declarant_prenom}")
            print(f"Declarant Email: {self.general_info.declarant_email}, Declarant Telephone: {self.general_info.declarant_telephone}, Declarant Date of Birth: {self.general_info.declarant_date_naissance}")
            print(f"Declarant Address: {self.general_info.declarant_voie}, Address Complement: {self.general_info.declarant_complement}, Postal Code: {self.general_info.declarant_code_postal}, City: {self.general_info.declarant_ville}, Country: {self.general_info.declarant_pays}")


        print("Attached Files:")
        for file in self.attached_files:
            print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")

        print("Professional Activities:")
        for activity in self.professional_activities:
            print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
            for rem in activity.remuneration:
                print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

        print("Consultant Activities:")
        for activity in self.consultant_activities:
            print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Employer: {activity.employer_name}, Description: {activity.activity_description}, Start Date: {activity.start_date}, End Date: {activity.end_date}, Remuneration Type: {activity.remuneration_type}, Amount: {activity.remuneration_amount}, Year: {activity.remuneration_year}")

        print("Volunteer Functions:")
        for function in self.volunteer_functions:
            print(f"ID: {function.id}, Label: {function.label}, Comment: {function.comment}, Conserved: {function.conserved}, Organization Name: {function.organization_name}, Activity Description: {function.activity_description}")

        print("Elective Mandates:")
        for mandate in self.elective_mandates:
            print(f"ID: {mandate.id}, Label: {mandate.label}, Comment: {mandate.comment}, Conserved: {mandate.conserved}, Description: {mandate.description}, Start Date: {mandate.start_date}, End Date: {mandate.end_date}")
            for rem in mandate.remuneration:
                print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

        print("Spouse's Professional Activities:")
        for activity in self.spouse_activities:
            print(f"ID: {activity.id}, Label: {activity.label}, Comment: {activity.comment}, Spouse Name: {activity.spouse_name}, Employer: {activity.employer}, Activity Description: {activity.activity_description}")

        print("Directing Participation")
        for participation in self.director_participations:
            print(f"Participation ID: {participation.id}, Label: {participation.label}, Comment: {participation.comment}, Conserved: {participation.conserved}, Company Name: {participation.company_name}, Description: {participation.activity_description}, Start Date: {participation.start_date}, End Date: {participation.end_date}")
            for rem in participation.remuneration:
                print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

        print("Financial Participations:")
        for participation in self.financial_participations:
            print(f"ID: {participation.id}, Label: {participation.label}, Comment: {participation.comment}, Company Name: {participation.company_name}, Evaluation: {participation.evaluation}, Remuneration: {participation.remuneration}, Capital Held: {participation.capital_held}, Number of Shares: {participation.number_of_shares}")

        print("Parliamentary Collaborators:")
        for collaborator in self.parliamentary_collaborators:
            print(f"ID: {collaborator.id}, Label: {collaborator.label}, Comment: {collaborator.comment}, Collaborator Name: {collaborator.collaborator_name}, Employer: {collaborator.employer}, Activity Description: {collaborator.activity_description}")

        print("Observations:")
        for observation in self.observations:
            print(f"ID: {observation.id}, Label: {observation.label}, Comment: {observation.comment}, Content: {observation.content}")


    def markdown_info(self):
        """
        Output detailed information about the Declaration in Markdown format, including observations.
        """
        markdown_output = "# Declaration Information\n\n"
        markdown_output += "## General Information\n"
        if self.general_info:
            markdown_output += f"- **Type of Declaration ID:** {self.general_info.type_declaration_id}, **Label:** {self.general_info.type_declaration_label}\n"
            markdown_output += f"- **Label Mandate:** {self.general_info.mandat_label}, **Type of Mandate:** {self.general_info.type_mandat}\n"
            markdown_output += f"- **Cod Category Mandate:** {self.general_info.cod_categorie_mandat}, **Name Category Mandate:** {self.general_info.nom_categorie_mandat}\n"
            markdown_output += f"- **Cod Type Mandate File:** {self.general_info.cod_type_mandat_fichier}, **Label Type Mandate:** {self.general_info.label_type_mandat}\n"
            markdown_output += f"- **Label Organ:** {self.general_info.label_organe}, **Code List Organ:** {self.general_info.code_liste_organe}\n"
            markdown_output += f"- **Help Text:** {self.general_info.texte_aide}\n"
            markdown_output += f"- **Organ Code:** {self.general_info.code_organe}, **Organ Name:** {self.general_info.nom_organe}, **Organ Label:** {self.general_info.label_organe}, **Declaration Label:** {self.general_info.label_declaration}\n"
            markdown_output += f"- **Declarant Quality:** {self.general_info.qualite_declarant}, **Declarant Quality for PDF:** {self.general_info.qualite_declarant_pdf}\n"
            markdown_output += f"- **Start Date Mandate:** {self.general_info.date_debut_mandat}, **End Date Mandate:** {self.general_info.date_fin_mandat}, **Last Declaration Date:** {self.general_info.date_derniere_declaration}\n"
            markdown_output += f"- **Marital Regime:** {self.general_info.regime_matrimonial}, **Marital Regime Comments:** {self.general_info.regime_matrimonial_comments}\n"
            markdown_output += f"- **Company Name:** {self.general_info.nom_societe}, **Parent Company Name:** {self.general_info.nom_societe_mere}, **Turnover:** {self.general_info.chiffre_affaire}, **Number of Dwellings:** {self.general_info.nombre_logements}\n"
            markdown_output += f"- **Declarant Civility:** {self.general_info.declarant_civilite}, **Declarant Name:** {self.general_info.declarant_nom}, **Declarant First Name:** {self.general_info.declarant_prenom}\n"
            markdown_output += f"- **Declarant Email:** {self.general_info.declarant_email}, **Declarant Telephone:** {self.general_info.declarant_telephone}, **Declarant Date of Birth:** {self.general_info.declarant_date_naissance}\n"
            markdown_output += f"- **Declarant Address:** {self.general_info.declarant_voie}, **Address Complement:** {self.general_info.declarant_complement}, **Postal Code:** {self.general_info.declarant_code_postal}, **City:** {self.general_info.declarant_ville}, **Country:** {self.general_info.declarant_pays}\n"

        markdown_output += "\n## Attached Files\n"
        for file in self.attached_files:
            markdown_output += f"- **File Name:** {file.file_name}, **Server File Name:** {file.server_file_name}\n"

        markdown_output += "\n## Professional Activities\n"
        for activity in self.professional_activities:
            markdown_output += f"- **ID:** {activity.id}, **Label:** {activity.label}, **Comment:** {activity.comment}, **Conserved:** {activity.conserved}, **Description:** {activity.description}, **Employer Name:** {activity.employer_name}, **Start Date:** {activity.start_date}, **End Date:** {activity.end_date}\n"
            for rem in activity.remuneration:
                markdown_output += f"  - **Remuneration Type:** {rem.type}, **Year:** {rem.year}, **Amount:** {rem.amount}\n"

        markdown_output += "\n## Consultant Activities\n"
        for activity in self.consultant_activities:
            markdown_output += f"- **ID:** {activity.id}, **Label:** {activity.label}, **Comment:** {activity.comment}, **Employer:** {activity.employer_name}, **Description:** {activity.activity_description}, **Start Date:** {activity.start_date}, **End Date:** {activity.end_date}, **Remuneration Type:** {activity.remuneration_type}, **Amount:** {activity.remuneration_amount}, **Year:** {activity.remuneration_year}\n"

        markdown_output += "\n## Volunteer Functions\n"
        for function in self.volunteer_functions:
            markdown_output += f"- **ID:** {function.id}, **Label:** {function.label}, **Comment:** {function.comment}, **Conserved:** {function.conserved}, **Organization Name:** {function.organization_name}, **Activity Description:** {function.activity_description}\n"

        markdown_output += "\n## Elective Mandates\n"
        for mandate in self.elective_mandates:
            markdown_output += f"- **ID:** {mandate.id}, **Label:** {mandate.label}, **Comment:** {mandate.comment}, **Conserved:** {mandate.conserved}, **Description:** {mandate.description}, **Start Date:** {mandate.start_date}, **End Date:** {mandate.end_date}\n"
            for rem in mandate.remuneration:
                markdown_output += f"  - **Remuneration Type:** {rem.type}, **Year:** {rem.year}, **Amount:** {rem.amount}\n"

        markdown_output += "\n## Spouse's Professional Activities\n"
        for activity in self.spouse_activities:
            markdown_output += f"- **ID:** {activity.id}, **Label:** {activity.label}, **Comment:** {activity.comment}, **Spouse Name:** {activity.spouse_name}, **Employer:** {activity.employer}, **Activity Description:** {activity.activity_description}\n"

        markdown_output += "\n## Directing Participation\n"
        for participation in self.director_participations:
            markdown_output += f"- **Participation ID:** {participation.id}, **Label:** {participation.label}, **Comment:** {participation.comment}, **Conserved:** {participation.conserved}, **Company Name:** {participation.company_name}, **Description:** {participation.activity_description}, **Start Date:** {participation.start_date}, **End Date:** {participation.end_date}\n"
            for rem in participation.remuneration:
                markdown_output += f"  - **Remuneration Type:** {rem.type}, **Year:** {rem.year}, **Amount:** {rem.amount}\n"

        markdown_output += "\n## Financial Participations\n"
        for participation in self.financial_participations:
            markdown_output += f"- **ID:** {participation.id}, **Label:** {participation.label}, **Comment:** {participation.comment}, **Company Name:** {participation.company_name}, **Evaluation:** {participation.evaluation}, **Remuneration:** {participation.remuneration}, **Capital Held:** {participation.capital_held}, **Number of Shares:** {participation.number_of_shares}\n"

        markdown_output += "\n## Parliamentary Collaborators\n"
        for collaborator in self.parliamentary_collaborators:
            markdown_output += f"- **ID:** {collaborator.id}, **Label:** {collaborator.label}, **Comment:** {collaborator.comment}, **Collaborator Name:** {collaborator.collaborator_name}, **Employer:** {collaborator.employer}, **Activity Description:** {collaborator.activity_description}\n"

        markdown_output += "\n## Observations\n"
        for observation in self.observations:
            markdown_output += f"- **ID:** {observation.id}, **Label:** {observation.label}, **Comment:** {observation.comment}, **Content:** {observation.content}\n"

        return markdown_output


# Example usage:
try:
    xml_string = singular_test_declaration  # Replace with valid XML string
    declaration_instance = Declaration(xml_string)
    declaration_instance.print_info()
except ET.ParseError:
    print("Failed to parse XML.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Declaration Information:

General Information:
Type of Declaration ID: DIA, Label: Déclaration d'intérêts et d'activités
Label Mandate: Député ou sénateur, Type of Mandate: Député
Cod Category Mandate: PAR, Name Category Mandate: Député ou sénateur
Cod Type Mandate File: depute, Label Type Mandate: Député
Label Organ: Ain(01), Code List Organ: ZEP
Help Text: None
Organ Code: ZEP, Organ Name: Zones d'élection des parlementaires, Organ Label: Ain(01), Declaration Label: None
Declarant Quality: None, Declarant Quality for PDF: Député/Ain(01)
Start Date Mandate: 19/06/2022, End Date Mandate: None, Last Declaration Date: None
Marital Regime: None, Marital Regime Comments: None
Company Name: None, Parent Company Name: None, Turnover: None, Number of Dwellings: None
Declarant Civility: M., Declarant Name: ABAD, Declarant First Name: DAMIEN
Declarant Email:             [Données non publiées]        , Declarant Telephone: None, Declarant Date of Birth: 05/04/1980
Declarant Address:             

In [ ]:
singular_test_declaration = declaration_ds.select(range(1000)).to_pandas().declaration_xml.to_list()[417]
singular_test_declaration

"<declaration><dateDepot>13/10/2020 13:44:50</dateDepot><uuid>b7f59856-debc-43a4-8da5-b6d9bc70adc2</uuid><origine>ADEL</origine><complete>true</complete><attachedFiles><attachedFiles><fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName><serverFileName /><base64EncodedContent /></attachedFiles></attachedFiles><declarationVersion>20171221</declarationVersion><activConsultantDto><neant>true</neant></activConsultantDto><activProfCinqDerniereDto><items><items><motif><id>CREATION</id><label /></motif><commentaire>76000/an</commentaire><description>PH Medecin urgentiste</description><employeur>chu dijon</employeur><remuneration><brutNet>Net</brutNet><montant><montant><annee>2014</annee><montant /></montant><montant><annee>2015</annee><montant /></montant><montant><annee>2016</annee><montant /></montant><montant><annee>2017</annee><montant /></montant><montant><annee>2018</annee><montant /></montant><montant><annee>2019</annee><montant /></montant></montant></remuneration><dateDebut>01/2014</

In [ ]:
from IPython.display import display_markdown

display_markdown(
    declaration_instance.markdown_info(),
    raw=True
)

# Declaration Information

## General Information
- **Type of Declaration ID:** DIA, **Label:** Déclaration d'intérêts et d'activités
- **Label Mandate:** Député ou sénateur, **Type of Mandate:** Député
- **Cod Category Mandate:** PAR, **Name Category Mandate:** Député ou sénateur
- **Cod Type Mandate File:** depute, **Label Type Mandate:** Député
- **Label Organ:** Ain(01), **Code List Organ:** ZEP
- **Help Text:** None
- **Organ Code:** ZEP, **Organ Name:** Zones d'élection des parlementaires, **Organ Label:** Ain(01), **Declaration Label:** None
- **Declarant Quality:** None, **Declarant Quality for PDF:** Député/Ain(01)
- **Start Date Mandate:** 19/06/2022, **End Date Mandate:** None, **Last Declaration Date:** None
- **Marital Regime:** None, **Marital Regime Comments:** None
- **Company Name:** None, **Parent Company Name:** None, **Turnover:** None, **Number of Dwellings:** None
- **Declarant Civility:** M., **Declarant Name:** ABAD, **Declarant First Name:** DAMIEN
- **Declarant Email:**             [Données non publiées]        , **Declarant Telephone:** None, **Declarant Date of Birth:** 05/04/1980
- **Declarant Address:**             [Données non publiées]        , **Address Complement:**             [Données non publiées]        , **Postal Code:**             [Données non publiées]        , **City:**             [Données non publiées]        , **Country:**             [Données non publiées]        

## Attached Files
- **File Name:** VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML, **Server File Name:** None

## Professional Activities

## Consultant Activities

## Volunteer Functions

## Elective Mandates

## Spouse's Professional Activities
- **ID:** CREATION, **Label:** None, **Comment:** None, **Spouse Name:**             [Données non publiées]        , **Employer:** CENTRE HOSPITALIER DU HAUT-BUGEY, **Activity Description:** Infirmière

## Directing Participation
- **Participation ID:** CREATION, **Label:** None, **Comment:** Président du SDIS 01. Activité arrêtée depuis ma démission de la présidence du département en juillet 2017, **Conserved:** None, **Company Name:** SDIS 01, **Description:** Service départemental d'incendie et de secours, **Start Date:** 04/2015, **End Date:** 07/2017
  - **Remuneration Type:** Brut, **Year:** [2015, 2016, 2017], **Amount:** [0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Président de l'association AINTOURISME qui développe et promeut la politique touristique du Département de l'Ain., **Conserved:** None, **Company Name:** ASSOCIATION AINTOURISME, **Description:** Président d'Aintourisme qui a pour objet la politique touristique du Département, **Start Date:** 07/2017, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2017, 2018, 2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Mon élection comme président du groupe LR à l'assemblée nationale date du 6 Novembre 2019. Ce titre n'octroie pas une rémunération supplémentaire., **Conserved:** None, **Company Name:** Assemblée nationale - Groupe Les Républicains -, **Description:** Président du Groupe Les Républicains à l'Assemblée nationale, **Start Date:** 11/2019, **End Date:** 05/2022
  - **Remuneration Type:** Net, **Year:** [2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Promotion du patrimoine culinaire et gastronomique du Département de l'Ain, **Conserved:** None, **Company Name:** ASSOCIATION SAVEURS DE L'AIN, **Description:** Président de l'association Saveurs de l'Ain qui a pour objet de promouvoir la gastronomie de notre Département, **Start Date:** 05/2019, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Association - parti politique Création Décembre 2018 Nous avons changé de noms à plusieurs reprises : d'abord la France des territoires, puis Dans l'Action puis Les Amis de Damien Abad, nouveau nom depuis Septembre 2020., **Conserved:** None, **Company Name:** Association Les Amis de Damien Abad, **Description:** Président de l'association Les Amis de Damien Abad, **Start Date:** 12/2018, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2018, 2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0, 0.0]

## Financial Participations
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** ORANGE, **Evaluation:** 877, **Remuneration:** néant, **Capital Held:** None, **Number of Shares:** 83
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** CREDIT AGRICOLE SA, **Evaluation:** 2910, **Remuneration:** néant, **Capital Held:** None, **Number of Shares:** 341
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** AIRBUS, **Evaluation:** 1929, **Remuneration:** NEANT, **Capital Held:** None, **Number of Shares:** 20
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** L'OREAL, **Evaluation:** 6552, **Remuneration:** NEANT, **Capital Held:** None, **Number of Shares:** 20

## Parliamentary Collaborators
- **ID:** CREATION, **Label:** None, **Comment:** None, **Collaborator Name:** LOPES MAGNUSON, **Employer:** Néant, **Activity Description:** Assistant fonctionnel
- **ID:** CREATION, **Label:** None, **Comment:** None, **Collaborator Name:** FOUGNIES REBECCA, **Employer:** Néant, **Activity Description:** Assistante parlementaire locale

## Observations
- **ID:** CREATION, **Label:** None, **Comment:** None, **Content:** mon équipe de collaborateurs parlementaires sera complétée dans les prochaines semaines


In [ ]:
len(declaration_instance.markdown_info())

5848

## Convert XML dataset to Markdown

In [ ]:
# un-comment to test a small sample
# small_sample = declaration_ds.select(range(100))
# small_sample

In [ ]:
def convert_xml_declaration_to_markdown(row):
  try:
      xml_string = row['declaration_xml']
      declaration_instance = Declaration(xml_string)
      # declaration_instance.print_info()
  except ET.ParseError:
      print("Failed to parse XML.")
  except Exception as e:
      print(f"An unexpected error occurred: {e}")

  declaration_markdown = declaration_instance.markdown_info()
  row['markdown_sha1'] = sha1_digest(declaration_markdown)
  row['declaration_markdown'] = declaration_markdown

  return row

In [ ]:
converted_ds = declaration_ds.map(convert_xml_declaration_to_markdown)

Map:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
converted_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'markdown_sha1', 'declaration_markdown'],
    num_rows: 10944
})

In [ ]:
converted_ds.select(range(10)).to_pandas()

,xml_sha1,declaration_xml,markdown_sha1,declaration_markdown
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,7897b9a3c34ee2dd9f6d743520f8ad78363d39cb,# Declaration Information\n\n## General Inform...
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,b0c12e2f087cc8c98b47d5440077899ee9cbb699,# Declaration Information\n\n## General Inform...
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,bc15183f3557c5fa34ea637a1e86cfafb13e3f9d,# Declaration Information\n\n## General Inform...
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,c71569daa76d9bfb929ab29b1cb57debaf7529bc,# Declaration Information\n\n## General Inform...
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,47c4c7aeeed69bd771964751aac442d3417c0baf,# Declaration Information\n\n## General Inform...
5,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/09/2021 10:41:48</d...,879a7cd01dff6c5223c6703d2a6d81a1aa1c2e6a,# Declaration Information\n\n## General Inform...
6,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>21/08/2021 19:39:06</d...,0a5687a4bd27dfa96ad94f291b79b9bf71c77781,# Declaration Information\n\n## General Inform...
7,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>23/08/2021 21:37:05</d...,0c4b23f721d9a3a072397e58dbdbf2d665749756,# Declaration Information\n\n## General Inform...
8,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/08/2021 00:09:05</d...,4a7037710fd8f254a18cbe4e6eaebdffa6becd33,# Declaration Information\n\n## General Inform...
9,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>28/09/2020 21:30:36</d...,9449fe3cf77835c8b1ce65ec132b0142c7777de2,# Declaration Information\n\n## General Inform...


In [ ]:
if create_dataset:
  converted_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_markdown")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

## Convert to SQL model using SQL Alchemy  

This part doesn't work and could work if I had 1000 years to set the correct types and learn how SQLAlchemy works.  

In [ ]:
# aaaa #create an error to stop the notebook

### declare base session (SQLite)

In [ ]:
# from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import relationship, sessionmaker
# from sqlalchemy.types import PickleType

# Base = declarative_base()

# # If you're using SQLite, you can use a database in memory or a file:
# engine = create_engine('sqlite:///declaration.db')
# Session = sessionmaker(bind=engine)
# session = Session()


### Test declaration -> insertion -> query

In [ ]:
# class Declaration(Base):
#     __tablename__ = 'declaration'
#     __table_args__ = {'extend_existing': True}  # Allow redefinition if needed
#     id = Column(Integer, primary_key=True)
#     uuid = Column(String)
#     date_depot = Column(Date)
#     origine = Column(String)
#     complete = Column(String)

#     # Relationships
#     general_info = relationship("GeneralInfo", uselist=False, back_populates="declaration")

# class GeneralInfo(Base):
#     __tablename__ = 'general_info'
#     __table_args__ = {'extend_existing': True}  # Allow redefinition if needed
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declaration.id'), unique=True)
#     type_declaration_id = Column(String)
#     type_declaration_label = Column(String)
#     mandat_label = Column(String)
#     type_mandat = Column(String)
#     date_debut_mandat = Column(Date)
#     date_fin_mandat = Column(Date)

#     # Back reference
#     declaration = relationship("Declaration", back_populates="general_info")


# # Add other models similarly...

# # Create all tables in the database
# Base.metadata.create_all(engine)


In [ ]:
# Base.metadata.drop_all(engine)  # Drop all tables
# Base.metadata.create_all(engine)  # Recreate tables with the new definitions

In [ ]:
# from datetime import datetime

# # Define a function to convert date strings to date objects
# def parse_date(date_str):
#     return datetime.strptime(date_str, '%Y-%m-%d').date()

# # Create an instance of the Declaration model with date conversion
# new_declaration = Declaration(
#     uuid="unique-identifier-12345",
#     date_depot=parse_date('2024-04-25'),  # Convert string to date object
#     origine="manual",
#     complete="yes"
# )

# # Create an instance of the GeneralInfo model with date conversion
# new_general_info = GeneralInfo(
#     type_declaration_id="001",
#     type_declaration_label="Initial Declaration",
#     mandat_label="Mayor",
#     type_mandat="Elective",
#     date_debut_mandat=parse_date('2020-01-01'),  # Convert string to date object
#     date_fin_mandat=parse_date('2024-01-01'),  # Convert string to date object
#     declaration=new_declaration  # Link this general info to the declaration we created
# )

# # Add both instances to the session
# session.add(new_declaration)
# session.add(new_general_info)

# # Commit the session to the database
# session.commit()


In [ ]:
# # Query for a specific declaration
# declaration_query = session.query(Declaration).filter(Declaration.uuid == "unique-identifier-12345").first()

# if declaration_query:
#     print("Found Declaration:", declaration_query.uuid, declaration_query.date_depot)

#     # Accessing linked GeneralInfo via relationship
#     general_info_query = session.query(GeneralInfo).filter(GeneralInfo.declaration_id == declaration_query.id).first()
#     if general_info_query:
#         print("General Info:")
#         print("Type of Declaration ID:", general_info_query.type_declaration_id)
#         print("Type of Declaration Label:", general_info_query.type_declaration_label)
#         print("Mandat Label:", general_info_query.mandat_label)
#         print("Type of Mandat:", general_info_query.type_mandat)
#         print("Date Start Mandat:", general_info_query.date_debut_mandat)
#         print("Date End Mandat:", general_info_query.date_fin_mandat)
# else:
#     print("No declaration found with the specified UUID.")


### Make complete SQLAlchemy model

In [ ]:
# from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date, Float, Boolean
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import relationship, backref
# from sqlalchemy.types import LargeBinary
# from sqlalchemy.orm import relationship

# Base = declarative_base()

# class AttachedFiles(Base):
#     __tablename__ = 'attached_files'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     file_name = Column(String)
#     server_file_name = Column(String)

#     declaration = relationship('Declaration', back_populates='attached_files')

#     def __init__(self, file_name, server_file_name):
#         self.file_name = file_name
#         self.server_file_name = server_file_name

#     def __repr__(self):
#         return f"<AttachedFiles(file_name='{self.file_name}', server_file_name='{self.server_file_name}')>"

# class ActivConsultantDto(Base):
#     __tablename__ = 'consultant_activities'
#     id = Column(Integer, primary_key=True)
#     motif_id = Column(String)
#     motif_label = Column(String)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     comment = Column(String)
#     conserved = Column(Boolean)
#     employer_name = Column(String)
#     activity_description = Column(String)
#     # temp disable date to move forward
#     # start_date = Column(Date)
#     # end_date = Column(Date)
#     start_date = Column(String)
#     end_date = Column(String)
#     remuneration_type = Column(String)
#     remuneration_amount = Column(Float)
#     remuneration_year = Column(Integer)

#     declaration = relationship('Declaration', back_populates='consultant_activities')

#     def __init__(self, motif_id, motif_label, comment, conserved, employer_name, activity_description, start_date, end_date, remuneration_type, remuneration_amount, remuneration_year):
#         self.motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.conserved = conserved
#         self.employer_name = employer_name
#         self.activity_description = activity_description
#         self.start_date = start_date
#         self.end_date = end_date
#         self.remuneration_type = remuneration_type
#         self.remuneration_amount = remuneration_amount
#         self.remuneration_year = remuneration_year

#     def __repr__(self):
#         return f"<ActivConsultantDto(label='{self.label}', employer_name='{self.employer_name}', start_date='{self.start_date}', end_date='{self.end_date}')>"


# class FonctionBenevoleDto(Base):
#     __tablename__ = 'volunteer_functions'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     conserved = Column(Boolean)
#     organization_name = Column(String)
#     activity_description = Column(String)

#     declaration = relationship('Declaration', back_populates='volunteer_functions')

#     def __init__(self,motif_id, motif_label, comment, conserved, organization_name, activity_description):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.conserved = conserved
#         self.organization_name = organization_name
#         self.activity_description = activity_description

#     def __repr__(self):
#         return f"<FonctionBenevoleDto(label='{self.label}', organization_name='{self.organization_name}')>"

# class ActivProfCinqDerniereDto(Base):
#     __tablename__ = 'professional_activities'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     conserved = Column(Boolean)
#     description = Column(String)
#     employer_name = Column(String)
#     # temp diable date to move forwards
#     # start_date = Column(Date)
#     # end_date = Column(Date)
#     start_date = Column(String)
#     end_date = Column(String)

#     declaration = relationship('Declaration', back_populates='professional_activities')

#     def __init__(self, motif_id, motif_label, comment, conserved, description, employer_name, start_date, end_date):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.conserved = conserved
#         self.description = description
#         self.employer_name = employer_name
#         self.start_date = start_date
#         self.end_date = end_date

#     def __repr__(self):
#         return f"<ActivProfCinqDerniereDto(label='{self.label}', employer_name='{self.employer_name}', start_date='{self.start_date}', end_date='{self.end_date}')>"

# class Remuneration(Base):
#     __tablename__ = 'remunerations'
#     id = Column(Integer, primary_key=True)
#     # activ_prof_cinq_derniere_id = Column(Integer, ForeignKey('activ_prof_cinq_dernieres.id'))
#     type = Column(String)
#     year = Column(Integer)
#     amount = Column(Float)

#     def __init__(self, type, year, amount):
#         self.type = type
#         self.year = year
#         self.amount = amount

#     # def __init__(self, activ_prof_cinq_derniere_id, type, year, amount):
#     #     self.activ_prof_cinq_derniere_id = activ_prof_cinq_derniere_id
#     #     self.type = type
#     #     self.year = year
#     #     self.amount = amount

#     def __repr__(self):
#         return f"<Remuneration(type='{self.type}', year={self.year}, amount={self.amount})>"

# class MandatElectifDto(Base):
#     __tablename__ = 'elective_mandates'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     conserved = Column(Boolean)
#     description = Column(String)
#     # temp diable to move forward - type date is an issue
#     # start_date = Column(Date)
#     # end_date = Column(Date)
#     start_date = Column(String)
#     end_date = Column(String)

#     declaration = relationship('Declaration', back_populates='elective_mandates')

#     def __init__(self, motif_id, motif_label, comment, conserved, description, start_date, end_date):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.conserved = conserved
#         self.description = description
#         self.start_date = start_date
#         self.end_date = end_date

#     def __repr__(self):
#         return f"<MandatElectifDto(label='{self.label}', description='{self.description}', start_date='{self.start_date}', end_date='{self.end_date}')>"

# class GeneralInfo(Base):
#     __tablename__ = 'general_info'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     type_declaration_id = Column(Integer)
#     type_declaration_label = Column(String)
#     mandat_label = Column(String)
#     type_mandat = Column(String)
#     cod_categorie_mandat = Column(String)
#     nom_categorie_mandat = Column(String)
#     cod_type_mandat_fichier = Column(String)
#     label_type_mandat = Column(String)
#     label_organe = Column(String)
#     code_liste_organe = Column(String)
#     texte_aide = Column(String)
#     code_organe = Column(String)
#     nom_organe = Column(String)
#     label_organe = Column(String)
#     label_declaration = Column(String)
#     qualite_declarant = Column(String)
#     qualite_declarant_pdf = Column(String)
#     # temp disable to move forwards
#     # date_debut_mandat = Column(Date)
#     # date_fin_mandat = Column(Date)
#     # date_derniere_declaration = Column(Date)
#     date_debut_mandat = Column(String)
#     date_fin_mandat = Column(String)
#     date_derniere_declaration = Column(String)

#     regime_matrimonial = Column(String)
#     regime_matrimonial_comments = Column(String)
#     nom_societe = Column(String)
#     nom_societe_mere = Column(String)
#     chiffre_affaire = Column(String)
#     nombre_logements = Column(Integer)
#     declarant_civilite = Column(String)
#     declarant_nom = Column(String)
#     declarant_prenom = Column(String)
#     declarant_email = Column(String)
#     declarant_telephone = Column(String)
#     # temp disable to move forwards
#     # declarant_date_naissance = Column(Date)
#     declarant_date_naissance = Column(String)
#     declarant_voie = Column(String)
#     declarant_complement = Column(String)
#     declarant_code_postal = Column(String)
#     declarant_ville = Column(String)
#     declarant_pays = Column(String)

#     declaration = relationship('Declaration', back_populates='general_info')

#     def __init__(self, type_declaration_id, type_declaration_label, mandat_label, type_mandat, cod_categorie_mandat, nom_categorie_mandat, cod_type_mandat_fichier, label_type_mandat, code_liste_organe, texte_aide, code_organe, nom_organe, label_organe, label_declaration, qualite_declarant, qualite_declarant_pdf, date_debut_mandat, date_fin_mandat, date_derniere_declaration, regime_matrimonial, regime_matrimonial_comments, nom_societe, nom_societe_mere, chiffre_affaire, nombre_logements, declarant_civilite, declarant_nom, declarant_prenom, declarant_email, declarant_telephone, declarant_date_naissance, declarant_voie, declarant_complement, declarant_code_postal, declarant_ville, declarant_pays):
#         self.type_declaration_id = type_declaration_id
#         self.type_declaration_label = type_declaration_label
#         self.mandat_label = mandat_label
#         self.type_mandat = type_mandat
#         self.cod_categorie_mandat = cod_categorie_mandat
#         self.nom_categorie_mandat = nom_categorie_mandat
#         self.cod_type_mandat_fichier = cod_type_mandat_fichier
#         self.label_type_mandat = label_type_mandat
#         self.code_liste_organe = code_liste_organe
#         self.texte_aide = texte_aide
#         self.code_organe = code_organe
#         self.nom_organe = nom_organe
#         self.label_organe = label_organe
#         self.label_declaration = label_declaration
#         self.qualite_declarant = qualite_declarant
#         self.qualite_declarant_pdf = qualite_declarant_pdf
#         self.date_debut_mandat = date_debut_mandat
#         self.date_fin_mandat = date_fin_mandat
#         self.date_derniere_declaration = date_derniere_declaration
#         self.regime_matrimonial = regime_matrimonial
#         self.regime_matrimonial_comments = regime_matrimonial_comments
#         self.nom_societe = nom_societe
#         self.nom_societe_mere = nom_societe_mere
#         self.chiffre_affaire = chiffre_affaire
#         self.nombre_logements = nombre_logements
#         self.declarant_civilite = declarant_civilite
#         self.declarant_nom = declarant_nom
#         self.declarant_prenom = declarant_prenom
#         self.declarant_email = declarant_email
#         self.declarant_telephone = declarant_telephone
#         self.declarant_date_naissance = declarant_date_naissance
#         self.declarant_voie = declarant_voie
#         self.declarant_complement = declarant_complement
#         self.declarant_code_postal = declarant_code_postal
#         self.declarant_ville = declarant_ville
#         self.declarant_pays = declarant_pays

#     def __repr__(self):
#         return f"<GeneralInfo(type_declaration_id='{self.type_declaration_id}', mandat_label='{self.mandat_label}', label_organe='{self.label_organe}')>"

# class ActivProfConjointDto(Base):
#     __tablename__ = 'spouse_activities'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     spouse_name = Column(String)
#     employer = Column(String)
#     activity_description = Column(String)

#     declaration = relationship('Declaration', back_populates='spouse_activities')

#     def __init__(self, motif_id, motif_label, comment, spouse_name, employer, activity_description):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.spouse_name = spouse_name
#         self.employer = employer
#         self.activity_description = activity_description

#     def __repr__(self):
#         return f"<ActivProfConjointDto(label='{self.label}', spouse_name='{self.spouse_name}', employer='{self.employer}')>"

# class ParticipationDirigeantDto(Base):
#     __tablename__ = 'director_participations'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     conserved = Column(Boolean)
#     company_name = Column(String)
#     activity_description = Column(String)
#     # temp disable to move forwards
#     # start_date = Column(Date)
#     # end_date = Column(Date)
#     start_date = Column(String)
#     end_date = Column(String)

#     declaration = relationship('Declaration', back_populates='director_participations')

#     def __init__(self, motif_id, motif_label, comment, conserved, company_name, activity_description, start_date, end_date):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.conserved = conserved
#         self.company_name = company_name
#         self.activity_description = activity_description
#         self.start_date = start_date
#         self.end_date = end_date

#     def __repr__(self):
#         return f"<ParticipationDirigeantDto(label='{self.label}', company_name='{self.company_name}', start_date='{self.start_date}', end_date='{self.end_date}')>"

# class ParticipationFinanciereDto(Base):
#     __tablename__ = 'financial_participations'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     company_name = Column(String)
#     evaluation = Column(String)
#     remuneration = Column(String)
#     capital_held = Column(Float)
#     number_of_shares = Column(Integer)

#     declaration = relationship('Declaration', back_populates='financial_participations')

#     def __init__(self, motif_id, motif_label, comment, company_name, evaluation, remuneration, capital_held, number_of_shares):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.company_name = company_name
#         self.evaluation = evaluation
#         self.remuneration = remuneration
#         self.capital_held = capital_held
#         self.number_of_shares = number_of_shares

#     def __repr__(self):
#         return f"<ParticipationFinanciereDto(label='{self.label}', company_name='{self.company_name}', evaluation='{self.evaluation}')>"

# class ActivCollaborateursDto(Base):
#     __tablename__ = 'parliamentary_collaborators'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     collaborator_name = Column(String)
#     employer = Column(String)
#     activity_description = Column(String)

#     declaration = relationship('Declaration', back_populates='parliamentary_collaborators')

#     def __init__(self, motif_id, motif_label, comment, collaborator_name, employer, activity_description):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.collaborator_name = collaborator_name
#         self.employer = employer
#         self.activity_description = activity_description

#     def __repr__(self):
#         return f"<ActivCollaborateursDto(label='{self.label}', collaborator_name='{self.collaborator_name}', employer='{self.employer}')>"

# class ObservationInteretDto(Base):
#     __tablename__ = 'observations'
#     id = Column(Integer, primary_key=True)
#     declaration_id = Column(Integer, ForeignKey('declarations.id'))
#     motif_id = Column(String)
#     motif_label = Column(String)
#     comment = Column(String)
#     content = Column(String)

#     declaration = relationship('Declaration', back_populates='observations')

#     def __init__(self, motif_id, motif_label, comment, content):
#         self.motif_id = motif_id
#         self.motif_label = motif_label
#         self.comment = comment
#         self.content = content

#     def __repr__(self):
#         return f"<ObservationInteretDto(label='{self.label}', content='{self.content}')>"

# class Declaration(Base):
#     __tablename__ = 'declarations'
#     id = Column(Integer, primary_key=True)
#     uuid = Column(String)
#     # temp disable to move forwards
#     # date_depot = Column(Date)
#     date_depot = Column(String)
#     origine = Column(String)
#     # temp diable to move forward
#     # complete = Column(Boolean)
#     complete = Column(String)

#     # Relationships
#     attached_files = relationship('AttachedFiles', back_populates='declaration')
#     professional_activities = relationship('ActivProfCinqDerniereDto', back_populates='declaration')
#     elective_mandates = relationship('MandatElectifDto', back_populates='declaration')
#     general_info = relationship('GeneralInfo', uselist=False, back_populates='declaration')
#     spouse_activities = relationship('ActivProfConjointDto', back_populates='declaration')
#     director_participations = relationship('ParticipationDirigeantDto', back_populates='declaration')
#     financial_participations = relationship('ParticipationFinanciereDto', back_populates='declaration')
#     parliamentary_collaborators = relationship('ActivCollaborateursDto', back_populates='declaration')
#     observations = relationship('ObservationInteretDto', back_populates='declaration')
#     consultant_activities = relationship('ActivConsultantDto', back_populates='declaration')
#     volunteer_functions = relationship('FonctionBenevoleDto', back_populates='declaration')

#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including observations.
#         """
#         print("Declaration Information:")

#         print(f"General Information:")
#         if self.general_info:
#             print(f"Type of Declaration ID: {self.general_info.type_declaration_id}, Label: {self.general_info.type_declaration_label}")
#             print(f"Label Mandate: {self.general_info.mandat_label}, Type of Mandate: {self.general_info.type_mandat}")
#             print(f"Cod Category Mandate: {self.general_info.cod_categorie_mandat}, Name Category Mandate: {self.general_info.nom_categorie_mandat}")
#             # Add more prints here for other fields as needed

#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")

#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
#             # for rem in activity.remuneration:
#                 # print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Consultant Activities:")
#         for activity in self.consultant_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Employer: {activity.employer_name}, Description: {activity.activity_description}, Start Date: {activity.start_date}, End Date: {activity.end_date}, Remuneration Type: {activity.remuneration_type}, Amount: {activity.remuneration_amount}, Year: {activity.remuneration_year}")

#         print("Volunteer Functions:")
#         for function in self.volunteer_functions:
#             print(f"ID: {function.motif_id}, Label: {function.motif_label}, Comment: {function.comment}, Conserved: {function.conserved}, Organization Name: {function.organization_name}, Activity Description: {function.activity_description}")

#         print("Elective Mandates:")
#         for mandate in self.elective_mandates:
#             print(f"ID: {mandate.motif_id}, Label: {mandate.motif_label}, Comment: {mandate.comment}, Conserved: {mandate.conserved}, Description: {mandate.description}, Start Date: {mandate.start_date}, End Date: {mandate.end_date}")
#             # for rem in mandate.remuneration:
#                 # print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Spouse's Professional Activities:")
#         for activity in self.spouse_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Spouse Name: {activity.spouse_name}, Employer: {activity.employer}, Activity Description: {activity.activity_description}")

#         print("Directing Participation")
#         for participation in self.director_participations:
#             print(f"Participation ID: {participation.motif_id}, Label: {participation.motif_label}, Comment: {participation.comment}, Conserved: {participation.conserved}, Company Name: {participation.company_name}, Description: {participation.activity_description}, Start Date: {participation.start_date}, End Date: {participation.end_date}")
#             # for rem in participation.remuneration:
#                 # print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Financial Participations:")
#         for participation in self.financial_participations:
#             print(f"ID: {participation.motif_id}, Label: {participation.motif_label}, Comment: {participation.comment}, Company Name: {participation.company_name}, Evaluation: {participation.evaluation}, Remuneration: {participation.remuneration}, Capital Held: {participation.capital_held}, Number of Shares: {participation.number_of_shares}")

#         print("Parliamentary Collaborators:")
#         for collaborator in self.parliamentary_collaborators:
#             print(f"ID: {collaborator.motif_id}, Label: {collaborator.motif_label}, Comment: {collaborator.comment}, Collaborator Name: {collaborator.collaborator_name}, Employer: {collaborator.employer}, Activity Description: {collaborator.activity_description}")

#         print("Observations:")
#         for observation in self.observations:
#             print(f"ID: {observation.motif_id}, Label: {observation.motif_label}, Comment: {observation.comment}, Content: {observation.content}")


# Base.metadata.create_all(engine)

### Complete integration of XML parsing and SQL insertion

We now transform the base classes that can parse XML into children of SQLA classes that are created using XML, but can still be manipulated with SQL.  

In [ ]:
# import xml.etree.ElementTree as ET
# from typing import List, Optional

# class AttachedFiles_xml(AttachedFiles):
#     def __init__(self, element):
#         """
#         Initialize the AttachedFiles class by parsing an XML element.

#         Attributes:
#         - file_name: "Nom du fichier de la pièce jointe"
#         - server_file_name: "Nom du fichier sur le serveur de la pièce jointe"
#         """
#         self.file_name = self._get_xml_text(element, 'fileName')
#         self.server_file_name = self._get_xml_text(element, 'serverFileName')
#         super().__init__(
#             file_name = self.file_name,
#             server_file_name = self.server_file_name
#         )

#     @staticmethod
#     def _get_xml_text(element, xpath):
#         """ Helper function to extract text from the specified XPath or return None if not found """
#         child = element.find(xpath)
#         if child is not None:
#             return child.text
#         else:
#             # Log a warning if the element is not found
#             print(f"Warning: No element found for XPath '{xpath}'. Returning None.")
#             return None

# class ActivConsultantDto_xml(ActivConsultantDto):
#     def __init__(self, element):
#         """
#         Initialize the ActivConsultantDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - employer_name: "Nom de l'employeur pour l'activité"
#         - activity_description: "Description de l'activité"
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         - remuneration_type: "Brut ou Net"
#         - remuneration_amount: "Montant de la rémunération"
#         - remuneration_year: "Année pour la rémunération"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.employer_name = element.find('nomEmployeur').text if element.find('nomEmployeur') is not None else None
#         self.activity_description = element.find('description').text if element.find('description') is not None else None
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None
#         remuneration_element = element.find('remuneration')
#         self.remuneration_type = remuneration_element.find('brutNet').text if remuneration_element is not None else None
#         self.remuneration_amount = remuneration_element.find('montant/montant').text if remuneration_element is not None else None
#         self.remuneration_year = remuneration_element.find('montant/annee').text if remuneration_element is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             conserved=self.conserved,
#             employer_name=self.employer_name,
#             activity_description=self.activity_description,
#             start_date=self.start_date,
#             end_date=self.end_date,
#             remuneration_type=self.remuneration_type,
#             remuneration_amount=self.remuneration_amount,
#             remuneration_year=self.remuneration_year
#         )

# class FonctionBenevoleDto_xml(FonctionBenevoleDto):
#     def __init__(self, element):
#         """
#         Initialize the FonctionBenevoleDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - organization_name: "Nom et objet de la structure ou de la personne morale"
#         - activity_description: "Description des activités et responsabilités exercées"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.organization_name = element.find('nomStructure').text if element.find('nomStructure') is not None else None
#         self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             conserved=self.conserved,
#             organization_name=self.organization_name,
#             activity_description=self.activity_description
#         )

# class ActivProfCinqDerniereDto_xml(ActivProfCinqDerniereDto):
#     def __init__(self, element):
#         """
#         Initialize the ActivProfCinqDerniereDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - description: "Description de l'activité"
#         - employer_name: "Nom de l'employeur pour l'activité"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.description = element.find('description').text if element.find('description') is not None else None
#         self.employer_name = element.find('employeur').text if element.find('employeur') is not None else None
#         self.remuneration = [Remuneration_xml(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             conserved=self.conserved,
#             description=self.description,
#             employer_name=self.employer_name,
#             remuneration=self.remuneration,
#             start_date=self.start_date,
#             end_date=self.end_date
#         )

# class Remuneration_xml(Remuneration):
#     def __init__(self, element):
#         """
#         Initialize the Remuneration class by parsing an XML element.

#         Attributes:
#         - type: "Brut ou Net"
#         - year: "Année pour la rémunération"
#         - amount: "Montant de la rémunération"
#         """
#         self.type = element.find('brutNet').text if element.find('brutNet') is not None else None
#         self.year = []
#         self.amount = []
#         # Handling multiple 'montant' elements properly
#         montants = element.findall('montant/montant')  # Adjusted XPath to access nested montant tags
#         for montant in montants:
#             year = montant.find('annee')
#             amount = montant.find('montant')
#             if year is not None and amount is not None:
#                 self.year.append(int(year.text))
#                 self.amount.append(self.parse_amount(amount.text))

#         super().__init__(
#             type=self.type,
#             year=self.year,
#             amount=self.amount
#         )

#     @staticmethod
#     def parse_amount(amount_str):
#         """ Parse and clean the amount string to handle non-numeric characters and convert it to float """
#         cleaned_amount = ''.join(c for c in amount_str if c.isdigit() or c == '.')
#         return float(cleaned_amount)

# class MandatElectifDto_xml(MandatElectifDto):
#     def __init__(self, element):
#         """
#         Initialize the MandatElectifDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - description: "Identification de la fonction élective ou du mandat"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.description = element.find('descriptionMandat').text if element.find('descriptionMandat') is not None else None
#         self.remuneration = [Remuneration_xml(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             conserved=self.conserved,
#             description=self.description,
#             # remuneration=self.remuneration, #temporary disable to move forward
#             start_date=self.start_date,
#             end_date=self.end_date
#         )

# class GeneralInfo_xml(GeneralInfo):
#     def __init__(self, element):
#         """
#         Initialize the GeneralInfo class by parsing an XML element.

#         Attributes: Various details about the declaration and the declarant as specified.
#         """
#         self.type_declaration_id = element.find('typeDeclaration/id').text if element.find('typeDeclaration/id') is not None else None
#         self.type_declaration_label = element.find('typeDeclaration/label').text if element.find('typeDeclaration/label') is not None else None
#         self.mandat_label = element.find('mandat/label').text if element.find('mandat/label') is not None else None
#         self.type_mandat = element.find('qualiteMandat/typeMandat').text if element.find('qualiteMandat/typeMandat') is not None else None
#         self.cod_categorie_mandat = element.find('qualiteMandat/codCategorieMandat').text if element.find('qualiteMandat/codCategorieMandat') is not None else None
#         self.nom_categorie_mandat = element.find('qualiteMandat/nomCategorieMandat').text if element.find('qualiteMandat/nomCategorieMandat') is not None else None
#         self.cod_type_mandat_fichier = element.find('qualiteMandat/codTypeMandatFichier').text if element.find('qualiteMandat/codTypeMandatFichier') is not None else None
#         self.label_type_mandat = element.find('qualiteMandat/labelTypeMandat').text if element.find('qualiteMandat/labelTypeMandat') is not None else None
#         # self.label_organe = element.find('qualiteMandat/labelOrgane').text if element.find('qualiteMandat/labelOrgane') is not None else None
#         self.code_liste_organe = element.find('qualiteMandat/codeListeOrgane').text if element.find('qualiteMandat/codeListeOrgane') is not None else None
#         self.texte_aide = element.find('qualiteMandat/texteAide').text if element.find('qualiteMandat/texteAide') is not None else None
#         self.code_organe = element.find('organe/codeListeOrgane').text if element.find('organe/codeListeOrgane') is not None else None
#         self.nom_organe = element.find('organe/nomListeOrgane').text if element.find('organe/nomListeOrgane') is not None else None
#         self.label_organe = element.find('organe/labelOrgane').text if element.find('organe/labelOrgane') is not None else None
#         self.label_declaration = element.find('organe/labelDeclaration').text if element.find('organe/labelDeclaration') is not None else None
#         self.qualite_declarant = element.find('qualiteDeclarant').text if element.find('qualiteDeclarant') is not None else None
#         self.qualite_declarant_pdf = element.find('qualiteDeclarantForPDF').text if element.find('qualiteDeclarantForPDF') is not None else None
#         self.date_debut_mandat = element.find('dateDebutMandat').text if element.find('dateDebutMandat') is not None else None
#         self.date_fin_mandat = element.find('dateFinMandat').text if element.find('dateFinMandat') is not None else None
#         self.date_derniere_declaration = element.find('dateDernDeclar').text if element.find('dateDernDeclar') is not None else None
#         self.regime_matrimonial = element.find('regimeMatrimonial').text if element.find('regimeMatrimonial') is not None else None
#         self.regime_matrimonial_comments = element.find('regimeMatrimonialComments').text if element.find('regimeMatrimonialComments') is not None else None
#         self.nom_societe = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.nom_societe_mere = element.find('nomSocieteMere').text if element.find('nomSocieteMere') is not None else None
#         self.chiffre_affaire = element.find('chiffreAffaire').text if element.find('chiffreAffaire') is not None else None
#         self.nombre_logements = element.find('nbLogements').text if element.find('nbLogements') is not None else None
#         self.declarant_civilite = element.find('declarant/civilite').text if element.find('declarant/civilite') is not None else None
#         self.declarant_nom = element.find('declarant/nom').text if element.find('declarant/nom') is not None else None
#         self.declarant_prenom = element.find('declarant/prenom').text if element.find('declarant/prenom') is not None else None
#         self.declarant_email = element.find('declarant/email').text if element.find('declarant/email') is not None else None
#         self.declarant_telephone = element.find('declarant/telephone').text if element.find('declarant/telephone') is not None else None
#         self.declarant_date_naissance = element.find('declarant/dateNaissance').text if element.find('declarant/dateNaissance') is not None else None
#         self.declarant_voie = element.find('declarant/adresseDec/voie').text if element.find('declarant/adresseDec/voie') is not None else None
#         self.declarant_complement = element.find('declarant/adresseDec/complement').text if element.find('declarant/adresseDec/complement') is not None else None
#         self.declarant_code_postal = element.find('declarant/adresseDec/codePostal').text if element.find('declarant/adresseDec/codePostal') is not None else None
#         self.declarant_ville = element.find('declarant/adresseDec/ville').text if element.find('declarant/adresseDec/ville') is not None else None
#         self.declarant_pays = element.find('declarant/adresseDec/pays').text if element.find('declarant/adresseDec/pays') is not None else None

#         super().__init__(
#             type_declaration_id=self.type_declaration_id,
#             type_declaration_label=self.type_declaration_label,
#             mandat_label=self.mandat_label,
#             type_mandat=self.type_mandat,
#             cod_categorie_mandat=self.cod_categorie_mandat,
#             nom_categorie_mandat=self.nom_categorie_mandat,
#             cod_type_mandat_fichier=self.cod_type_mandat_fichier,
#             label_type_mandat=self.label_type_mandat,
#             code_liste_organe=self.code_liste_organe,
#             texte_aide=self.texte_aide,
#             code_organe=self.code_organe,
#             nom_organe=self.nom_organe,
#             label_organe=self.label_organe,
#             label_declaration=self.label_declaration,
#             qualite_declarant=self.qualite_declarant,
#             qualite_declarant_pdf=self.qualite_declarant_pdf,
#             date_debut_mandat=self.date_debut_mandat,
#             date_fin_mandat=self.date_fin_mandat,
#             date_derniere_declaration=self.date_derniere_declaration,
#             regime_matrimonial=self.regime_matrimonial,
#             regime_matrimonial_comments=self.regime_matrimonial_comments,
#             nom_societe=self.nom_societe,
#             nom_societe_mere=self.nom_societe_mere,
#             chiffre_affaire=self.chiffre_affaire,
#             nombre_logements=self.nombre_logements,
#             declarant_civilite=self.declarant_civilite,
#             declarant_nom=self.declarant_nom,
#             declarant_prenom=self.declarant_prenom,
#             declarant_email=self.declarant_email,
#             declarant_telephone=self.declarant_telephone,
#             declarant_date_naissance=self.declarant_date_naissance,
#             declarant_voie=self.declarant_voie,
#             declarant_complement=self.declarant_complement,
#             declarant_code_postal=self.declarant_code_postal,
#             declarant_ville=self.declarant_ville,
#             declarant_pays=self.declarant_pays
#         )

# class ActivProfConjointDto_xml(ActivProfConjointDto):
#     def __init__(self, element):
#         """
#         Initialize the ActivProfConjointDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - spouse_name: "Nom du conjoint"
#         - employer: "Employeur du conjoint"
#         - activity_description: "Description de l’activité professionnelle du conjoint"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.spouse_name = element.find('nomConjoint').text if element.find('nomConjoint') is not None else None
#         self.employer = element.find('employeurConjoint').text if element.find('employeurConjoint') is not None else None
#         self.activity_description = element.find('activiteProf').text if element.find('activiteProf') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             spouse_name=self.spouse_name,
#             employer=self.employer,
#             activity_description=self.activity_description
#         )

# class ParticipationDirigeantDto_xml(ParticipationDirigeantDto):
#     def __init__(self, element):
#         """
#         Initialize the ParticipationDirigeantDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - conserved: "Indique si l'activité est conservée"
#         - company_name: "Identification de l’organisme public ou privé ou de la société"
#         - activity_description: "Description de l’activité"
#         - remuneration: List of Remuneration objects
#         - start_date: "Date de début de l'activité"
#         - end_date: "Date de fin de l'activité"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.conserved = element.find('conservee').text if element.find('conservee') is not None else None
#         self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.activity_description = element.find('activite').text if element.find('activite') is not None else None
#         self.remuneration = [Remuneration_xml(rem) for rem in element.findall('remuneration')]
#         self.start_date = element.find('dateDebut').text if element.find('dateDebut') is not None else None
#         self.end_date = element.find('dateFin').text if element.find('dateFin') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             conserved=self.conserved,
#             company_name=self.company_name,
#             activity_description=self.activity_description,
#             # remuneration=self.remuneration, #temp diable to move forward
#             start_date=self.start_date,
#             end_date=self.end_date
#         )

# class ParticipationFinanciereDto_xml(ParticipationFinanciereDto):
#     def __init__(self, element):
#         """
#         Initialize the ParticipationFinanciereDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - company_name: "Identification de la société"
#         - evaluation: "Évaluation de la participation financière"
#         - remuneration: "Rémunération ou gratification perçue"
#         - capital_held: "Pourcentage du capital détenu"
#         - number_of_shares: "Nombre de parts"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.company_name = element.find('nomSociete').text if element.find('nomSociete') is not None else None
#         self.evaluation = element.find('evaluation').text if element.find('evaluation') is not None else None
#         self.remuneration = element.find('remuneration').text if element.find('remuneration') is not None else None
#         self.capital_held = element.find('capitalDetenu').text if element.find('capitalDetenu') is not None else None
#         self.number_of_shares = element.find('nombreParts').text if element.find('nombreParts') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             company_name=self.company_name,
#             evaluation=self.evaluation,
#             remuneration=self.remuneration,
#             capital_held=self.capital_held,
#             number_of_shares=self.number_of_shares
#         )

# class ActivCollaborateursDto_xml(ActivCollaborateursDto):
#     def __init__(self, element):
#         """
#         Initialize the ActivCollaborateursDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - collaborator_name: "Nom et prénom du collaborateur"
#         - employer: "Identification des autres employeurs ou structures sociales d’emploi"
#         - activity_description: "Description des activités professionnelles"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.collaborator_name = element.find('nom').text if element.find('nom') is not None else None
#         self.employer = element.find('employeur').text if element.find('employeur') is not None else None
#         self.activity_description = element.find('descriptionActivite').text if element.find('descriptionActivite') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             collaborator_name=self.collaborator_name,
#             employer=self.employer,
#             activity_description=self.activity_description
#         )

# class ObservationInteretDto_xml(ObservationInteretDto):
#     def __init__(self, element):
#         """
#         Initialize the ObservationInteretDto class by parsing an XML element.

#         Attributes:
#         - id: "ID of the motif"
#         - label: "Label of the motif"
#         - comment: "Commentaire du déclarant sur la rubrique"
#         - content: "Observations du déclarant"
#         """
#         self.motif_id = element.find('motif/id').text if element.find('motif/id') is not None else None
#         self.motif_label = element.find('motif/label').text if element.find('motif/label') is not None else None
#         self.comment = element.find('commentaire').text if element.find('commentaire') is not None else None
#         self.content = element.find('contenu').text if element.find('contenu') is not None else None

#         super().__init__(
#             motif_id=self.motif_id,
#             motif_label=self.motif_label,
#             comment=self.comment,
#             content=self.content
#         )

# class Declaration_xml(Declaration):
#     def __init__(self, xml_string):
#         """
#         Initialize the Declaration class by parsing an XML string.

#         Attributes: Various details about the declaration including observations.
#         """
#         try:
#             xml_tree = ET.fromstring(xml_string)
#             self.date_depot = self._get_xml_text(xml_tree, 'dateDepot')
#             self.uuid = self._get_xml_text(xml_tree, 'uuid')
#             self.origine = self._get_xml_text(xml_tree, 'origine')
#             self.complete = self._get_xml_text(xml_tree, 'complete')
#             self.attached_files = self._parse_attached_files(xml_tree.find('attachedFiles'))
#             self.professional_activities = self._parse_professional_activities(xml_tree.find('activProfCinqDerniereDto'))
#             # print("all good! - 1")
#             self.elective_mandates = self._parse_elective_mandates(xml_tree.find('mandatElectifDto'))
#             # print("all good! - 2")
#             self.general_info = GeneralInfo_xml(xml_tree.find('general')) if xml_tree.find('general') is not None else None
#             self.spouse_activities = self._parse_spouse_activities(xml_tree.find('activProfConjointDto'))
#             self.director_participations = self._parse_director_participations(xml_tree.find('participationDirigeantDto'))
#             # print("all good! - 3")
#             self.financial_participations = self._parse_financial_participations(xml_tree.find('participationFinanciereDto'))
#             self.parliamentary_collaborators = self._parse_parliamentary_collaborators(xml_tree.find('activCollaborateursDto'))
#             # print("all good! - 4")
#             self.observations = self._parse_observations(xml_tree.find('observationInteretDto'))
#             self.consultant_activities = self._parse_consultant_activities(xml_tree.find('activConsultantDto'))
#             self.volunteer_functions = self._parse_volunteer_functions(xml_tree.find('fonctionBenevoleDto'))
#             # print("all good! - 5")

#             super().__init__(
#                 date_depot=self.date_depot,
#                 uuid=self.uuid,
#                 origine=self.origine,
#                 complete=self.complete,
#                 attached_files=self.attached_files,
#                 professional_activities=self.professional_activities,
#                 elective_mandates=self.elective_mandates,
#                 general_info=self.general_info,
#                 spouse_activities=self.spouse_activities,
#                 director_participations=self.director_participations,
#                 financial_participations=self.financial_participations,
#                 parliamentary_collaborators=self.parliamentary_collaborators,
#                 observations=self.observations,
#                 consultant_activities=self.consultant_activities,
#                 volunteer_functions=self.volunteer_functions
#             )
#         except ET.ParseError as e:
#             print(f"Error parsing XML: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

#     @staticmethod
#     def _get_xml_text(tree, xpath):
#         element = tree.find(xpath)
#         return element.text if element is not None else None

#     def _parse_attached_files(self, attached_files_element):
#         return [AttachedFiles_xml(file_element) for file_element in attached_files_element.findall('attachedFiles')] if attached_files_element else []

#     def _parse_professional_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfCinqDerniereDto_xml(item) for item in activities_element.findall('items/items')]

#     def _parse_elective_mandates(self, mandates_element):
#         if mandates_element is None:
#             return []  # No mandates to report or section is empty
#         return [MandatElectifDto_xml(item) for item in mandates_element.findall('items/items')]

#     def _parse_spouse_activities(self, activities_element):
#         if activities_element is None:
#             return []  # No activities to report or section is empty
#         return [ActivProfConjointDto_xml(item) for item in activities_element.findall('items/items')]

#     def _parse_director_participations(self, director_element):
#         if director_element is None:
#             return []  # No participations to report
#         return [ParticipationDirigeantDto_xml(item) for item in director_element.findall('items/items')]

#     def _parse_financial_participations(self, financial_element):
#         if financial_element is None:
#             return []  # No financial participations to report
#         return [ParticipationFinanciereDto_xml(item) for item in financial_element.findall('items/items')]

#     def _parse_parliamentary_collaborators(self, collaborators_element):
#         if collaborators_element is None:
#             return []  # No collaborators to report
#         return [ActivCollaborateursDto_xml(item) for item in collaborators_element.findall('items/items')]

#     def _parse_observations(self, observation_element):
#         if observation_element is None:
#             return []  # No observations to report
#         return [ObservationInteretDto_xml(item) for item in observation_element.findall('items/items')]

#     def _parse_consultant_activities(self, consultant_element):
#         if consultant_element is None:
#             return []  # No consultant activities to report
#         return [ActivConsultantDto_xml(item) for item in consultant_element.findall('items/items')]


#     def _parse_volunteer_functions(self, volunteer_element):
#         if volunteer_element is None:
#             return []  # No volunteer functions to report
#         return [FonctionBenevoleDto_xml(item) for item in volunteer_element.findall('items/items')]


#     def get_declaration(self):
#         return Declaration(
#             date_depot=self.date_depot,
#             uuid=self.uuid,
#             origine=self.origine,
#             complete=self.complete,
#             attached_files=self.attached_files,
#             professional_activities=self.professional_activities,
#             elective_mandates=self.elective_mandates,
#             general_info=self.general_info,
#             spouse_activities=self.spouse_activities,
#             director_participations=self.director_participations,
#             financial_participations=self.financial_participations,
#             parliamentary_collaborators=self.parliamentary_collaborators,
#             observations=self.observations,
#             consultant_activities=self.consultant_activities,
#             volunteer_functions=self.volunteer_functions
#         )


#     def print_info(self):
#         """
#         Print detailed information about the Declaration, including observations.
#         """
#         print("Declaration Information:")

#         print(f"General Information:")
#         if self.general_info:
#             print(f"Type of Declaration ID: {self.general_info.type_declaration_id}, Label: {self.general_info.type_declaration_label}")
#             print(f"Label Mandate: {self.general_info.mandat_label}, Type of Mandate: {self.general_info.type_mandat}")
#             print(f"Cod Category Mandate: {self.general_info.cod_categorie_mandat}, Name Category Mandate: {self.general_info.nom_categorie_mandat}")
#             # Add more prints here for other fields as needed

#         print("Attached Files:")
#         for file in self.attached_files:
#             print(f"File Name: {file.file_name}, Server File Name: {file.server_file_name}")

#         print("Professional Activities:")
#         for activity in self.professional_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Conserved: {activity.conserved}, Description: {activity.description}, Employer Name: {activity.employer_name}, Start Date: {activity.start_date}, End Date: {activity.end_date}")
#             for rem in activity.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Consultant Activities:")
#         for activity in self.consultant_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Employer: {activity.employer_name}, Description: {activity.activity_description}, Start Date: {activity.start_date}, End Date: {activity.end_date}, Remuneration Type: {activity.remuneration_type}, Amount: {activity.remuneration_amount}, Year: {activity.remuneration_year}")

#         print("Volunteer Functions:")
#         for function in self.volunteer_functions:
#             print(f"ID: {function.motif_id}, Label: {function.motif_label}, Comment: {function.comment}, Conserved: {function.conserved}, Organization Name: {function.organization_name}, Activity Description: {function.activity_description}")

#         print("Elective Mandates:")
#         for mandate in self.elective_mandates:
#             print(f"ID: {mandate.motif_id}, Label: {mandate.motif_label}, Comment: {mandate.comment}, Conserved: {mandate.conserved}, Description: {mandate.description}, Start Date: {mandate.start_date}, End Date: {mandate.end_date}")
#             for rem in mandate.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Spouse's Professional Activities:")
#         for activity in self.spouse_activities:
#             print(f"ID: {activity.motif_id}, Label: {activity.motif_label}, Comment: {activity.comment}, Spouse Name: {activity.spouse_name}, Employer: {activity.employer}, Activity Description: {activity.activity_description}")

#         print("Directing Participation")
#         for participation in self.director_participations:
#             print(f"Participation ID: {participation.motif_id}, Label: {participation.motif_label}, Comment: {participation.comment}, Conserved: {participation.conserved}, Company Name: {participation.company_name}, Description: {participation.activity_description}, Start Date: {participation.start_date}, End Date: {participation.end_date}")
#             for rem in participation.remuneration:
#                 print(f"Remuneration Type: {rem.type}, Year: {rem.year}, Amount: {rem.amount}")

#         print("Financial Participations:")
#         for participation in self.financial_participations:
#             print(f"ID: {participation.motif_id}, Label: {participation.motif_label}, Comment: {participation.comment}, Company Name: {participation.company_name}, Evaluation: {participation.evaluation}, Remuneration: {participation.remuneration}, Capital Held: {participation.capital_held}, Number of Shares: {participation.number_of_shares}")

#         print("Parliamentary Collaborators:")
#         for collaborator in self.parliamentary_collaborators:
#             print(f"ID: {collaborator.motif_id}, Label: {collaborator.motif_label}, Comment: {collaborator.comment}, Collaborator Name: {collaborator.collaborator_name}, Employer: {collaborator.employer}, Activity Description: {collaborator.activity_description}")

#         print("Observations:")
#         for observation in self.observations:
#             print(f"ID: {observation.motif_id}, Label: {observation.motif_label}, Comment: {observation.comment}, Content: {observation.content}")




In [ ]:
# # Example usage:
# try:
#     xml_string = singular_test_declaration  # Replace with valid XML string
#     declaration_instance = Declaration_xml(xml_string)
#     declaration_instance.print_info()
# except ET.ParseError:
#     print("Failed to parse XML.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")

In [ ]:
# # We have to get the parent class otherwise SQLA will give us hell
# session.add(declaration_instance.get_declaration())

# # Commit the transaction
# session.commit()

In [ ]:
# declarations = session.query(Declaration).all()

In [ ]:
# declarations[3].print_info()

In [ ]:
# for declaration in declarations:
#     print("UUID:", declaration.uuid)
#     print("Date Depot:", declaration.date_depot)
#     print("Origine:", declaration.origine)
#     print("Complete:", declaration.complete)

#     # Access associated records
#     for attached_file in declaration.attached_files:
#         print("Attached File:", attached_file.file_name, attached_file.server_file_name)

#     for professional_activity in declaration.professional_activities:
#         print("Professional Activity Label:", professional_activity.motif_id)
#         print("Professional Activity Label:", professional_activity.motif_label)
#         print("Professional Activity Description:", professional_activity.description)
#         print("Professional Activity Start Date:", professional_activity.start_date)
#         print("Professional Activity End Date:", professional_activity.end_date)

Once again we have a failure....

In [ ]:
# declarations

In [ ]:
# declarations[0].print_info()